In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 22 20:09:58 2021

@author: ronguy
"""
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

import numpy as np
import matplotlib

import matplotlib.pyplot as plt

import time
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.cluster import KMeans
import umap
from sklearn.cluster import DBSCAN
from sklearn import metrics

from tqdm import tqdm_notebook
from lmfit import minimize, Parameters
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as pl


# define a bunch of functions 

In [ ]:
def wfall(shap_values, max_display=10, show=True):
    """ Plots an explantion of a single prediction as a waterfall plot.
    The SHAP value of a feature represents the impact of the evidence provided by that feature on the model's
    output. The waterfall plot is designed to visually display how the SHAP values (evidence) of each feature
    move the model output from our prior expectation under the background data distribution, to the final model
    prediction given the evidence of all the features. Features are sorted by the magnitude of their SHAP values
    with the smallest magnitude features grouped together at the bottom of the plot when the number of features
    in the models exceeds the max_display parameter.
    
    Parameters
    ----------
    shap_values : Explanation
        A one-dimensional Explanation object that contains the feature values and SHAP values to plot.
    max_display : str
        The maximum number of features to plot.
    show : bool
        Whether matplotlib.pyplot.show() is called before returning. Setting this to False allows the plot
        to be customized further after it has been created.
    """
    dark_o= mpl.colors.to_rgb('dimgray')
    dim_g= mpl.colors.to_rgb('darkorange')

    base_values = shap_values.base_values
    
    features = shap_values.data
    feature_names = shap_values.feature_names
    lower_bounds = getattr(shap_values, "lower_bounds", None)
    upper_bounds = getattr(shap_values, "upper_bounds", None)
    values = shap_values.values

    # make sure we only have a single output to explain
    if (type(base_values) == np.ndarray and len(base_values) > 0) or type(base_values) == list:
        raise Exception("waterfall_plot requires a scalar base_values of the model output as the first " \
                        "parameter, but you have passed an array as the first parameter! " \
                        "Try shap.waterfall_plot(explainer.base_values[0], values[0], X[0]) or " \
                        "for multi-output models try " \
                        "shap.waterfall_plot(explainer.base_values[0], values[0][0], X[0]).")

    # make sure we only have a single explanation to plot
    if len(values.shape) == 2:
        raise Exception("The waterfall_plot can currently only plot a single explanation but a matrix of explanations was passed!")
    
    # unwrap pandas series
    if safe_isinstance(features, "pandas.core.series.Series"):
        if feature_names is None:
            feature_names = list(features.index)
        features = features.values

    # fallback feature names
    if feature_names is None:
        feature_names = np.array([labels['FEATURE'] % str(i) for i in range(len(values))])
    
    # init variables we use for tracking the plot locations
    num_features = min(max_display, len(values))
    row_height = 0.5
    rng = range(num_features - 1, -1, -1)
    order = np.argsort(-np.abs(values))
    pos_lefts = []
    pos_inds = []
    pos_widths = []
    pos_low = []
    pos_high = []
    neg_lefts = []
    neg_inds = []
    neg_widths = []
    neg_low = []
    neg_high = []
    loc = base_values + values.sum()
    yticklabels = ["" for i in range(num_features + 1)]
    
    # size the plot based on how many features we are plotting
    pl.gcf().set_size_inches(8, num_features * row_height + 1.5)

    # see how many individual (vs. grouped at the end) features we are plotting
    if num_features == len(values):
        num_individual = num_features
    else:
        num_individual = num_features - 1

    # compute the locations of the individual features and plot the dashed connecting lines
    for i in range(num_individual):
        sval = values[order[i]]
        loc -= sval
        if sval >= 0:
            pos_inds.append(rng[i])
            pos_widths.append(sval)
            if lower_bounds is not None:
                pos_low.append(lower_bounds[order[i]])
                pos_high.append(upper_bounds[order[i]])
            pos_lefts.append(loc)
        else:
            neg_inds.append(rng[i])
            neg_widths.append(sval)
            if lower_bounds is not None:
                neg_low.append(lower_bounds[order[i]])
                neg_high.append(upper_bounds[order[i]])
            neg_lefts.append(loc)
        if num_individual != num_features or i + 4 < num_individual:
            pl.plot([loc, loc], [rng[i] -1 - 0.4, rng[i] + 0.4], color="#bbbbbb", linestyle="--", linewidth=0.5, zorder=-1)
        if features is None:
            yticklabels[rng[i]] = feature_names[order[i]]
        else:
            yticklabels[rng[i]] = format_value(features[order[i]], "%0.03f") + " = " + feature_names[order[i]] 
    
    # add a last grouped feature to represent the impact of all the features we didn't show
    if num_features < len(values):
        yticklabels[0] = "%d other features" % (len(values) - num_features + 1)
        remaining_impact = base_values - loc
        if remaining_impact < 0:
            pos_inds.append(0)
            pos_widths.append(-remaining_impact)
            pos_lefts.append(loc + remaining_impact)
            c = dim_g  #colors.red_rgb
        else:
            neg_inds.append(0)
            neg_widths.append(-remaining_impact)
            neg_lefts.append(loc + remaining_impact)
            c = dark_o #colors.blue_rgb

    points = pos_lefts + list(np.array(pos_lefts) + np.array(pos_widths)) + neg_lefts + list(np.array(neg_lefts) + np.array(neg_widths))
    dataw = np.max(points) - np.min(points)
    
    # draw invisible bars just for sizing the axes
    label_padding = np.array([0.1*dataw if w < 1 else 0 for w in pos_widths])
    pl.barh(pos_inds, np.array(pos_widths) + label_padding + 0.02*dataw, left=np.array(pos_lefts) - 0.01*dataw, color=colors.red_rgb, alpha=0)
    label_padding = np.array([-0.1*dataw  if -w < 1 else 0 for w in neg_widths])
    pl.barh(neg_inds, np.array(neg_widths) + label_padding - 0.02*dataw, left=np.array(neg_lefts) + 0.01*dataw, color=colors.blue_rgb, alpha=0)
    
    # define variable we need for plotting the arrows
    head_length = 0.08
    bar_width = 0.8
    xlen = pl.xlim()[1] - pl.xlim()[0]
    fig = pl.gcf()
    ax = pl.gca()
    xticks = ax.get_xticks()
    bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    width, height = bbox.width, bbox.height
    bbox_to_xscale = xlen/width
    hl_scaled = bbox_to_xscale * head_length
    renderer = fig.canvas.get_renderer()
    
    # draw the positive arrows
    for i in range(len(pos_inds)):
        dist = pos_widths[i]
        arrow_obj = pl.arrow(
            pos_lefts[i], pos_inds[i], max(dist-hl_scaled, 0.000001), 0,
            head_length=min(dist, hl_scaled),
            color=dim_g, width=bar_width,
            head_width=bar_width
        )
        
        if pos_low is not None and i < len(pos_low):
            pl.errorbar(
                pos_lefts[i] + pos_widths[i], pos_inds[i], 
                xerr=np.array([[pos_widths[i] - pos_low[i]], [pos_high[i] - pos_widths[i]]]),
                ecolor=dim_g
            )

        txt_obj = pl.text(
            pos_lefts[i] + 0.5*dist, pos_inds[i], format_value(pos_widths[i], '%+0.02f'),
            horizontalalignment='center', verticalalignment='center', color="white",
            fontsize=12
        )
        text_bbox = txt_obj.get_window_extent(renderer=renderer)
        arrow_bbox = arrow_obj.get_window_extent(renderer=renderer)
        
        # if the text overflows the arrow then draw it after the arrow
        if text_bbox.width > arrow_bbox.width: 
            txt_obj.remove()
            
            txt_obj = pl.text(
                pos_lefts[i] + (5/72)*bbox_to_xscale + dist, pos_inds[i], format_value(pos_widths[i], '%+0.02f'),
                horizontalalignment='left', verticalalignment='center', color=dim_g,
                fontsize=12
            )
    
    # draw the negative arrows
    for i in range(len(neg_inds)):
        dist = neg_widths[i]
        
        arrow_obj = pl.arrow(
            neg_lefts[i], neg_inds[i], -max(-dist-hl_scaled, 0.000001), 0,
            head_length=min(-dist, hl_scaled),
            color=dark_o, width=bar_width,
            head_width=bar_width
        )

        if neg_low is not None and i < len(neg_low):
            pl.errorbar(
                neg_lefts[i] + neg_widths[i], neg_inds[i], 
                xerr=np.array([[neg_widths[i] - neg_low[i]], [neg_high[i] - neg_widths[i]]]),
                ecolor=dark_o
            )
        
        txt_obj = pl.text(
            neg_lefts[i] + 0.5*dist, neg_inds[i], format_value(neg_widths[i], '%+0.02f'),
            horizontalalignment='center', verticalalignment='center', color="white",
            fontsize=12
        )
        text_bbox = txt_obj.get_window_extent(renderer=renderer)
        arrow_bbox = arrow_obj.get_window_extent(renderer=renderer)
        
        # if the text overflows the arrow then draw it after the arrow
        if text_bbox.width > arrow_bbox.width: 
            txt_obj.remove()
            
            txt_obj = pl.text(
                neg_lefts[i] - (5/72)*bbox_to_xscale + dist, neg_inds[i], format_value(neg_widths[i], '%+0.02f'),
                horizontalalignment='right', verticalalignment='center', color=dark_o,
                fontsize=12
            )

    # draw the y-ticks twice, once in gray and then again with just the feature names in black
    ytick_pos = list(range(num_features)) + list(np.arange(num_features)+1e-8) # The 1e-8 is so matplotlib 3.3 doesn't try and collapse the ticks
    pl.yticks(ytick_pos, yticklabels[:-1] + [l.split('=')[-1] for l in yticklabels[:-1]], fontsize=13)
    
    # put horizontal lines for each feature row
    for i in range(num_features):
        pl.axhline(i, color="#cccccc", lw=0.5, dashes=(1, 5), zorder=-1)
    
    # mark the prior expected value and the model prediction
    pl.axvline(base_values, 0, 1/num_features, color="#bbbbbb", linestyle="--", linewidth=0.5, zorder=-1)
    fx = base_values + values.sum()
    pl.axvline(fx, 0, 1, color="#bbbbbb", linestyle="--", linewidth=0.5, zorder=-1)
    
    # clean up the main axis
    pl.gca().xaxis.set_ticks_position('bottom')
    pl.gca().yaxis.set_ticks_position('none')
    pl.gca().spines['right'].set_visible(False)
    pl.gca().spines['top'].set_visible(False)
    pl.gca().spines['left'].set_visible(False)
    ax.tick_params(labelsize=13)
    #pl.xlabel("\nModel output", fontsize=12)

    # draw the E[f(X)] tick mark
    xmin,xmax = ax.get_xlim()
    ax2=ax.twiny()
    ax2.set_xlim(xmin,xmax)
    ax2.set_xticks([base_values, base_values+1e-8]) # The 1e-8 is so matplotlib 3.3 doesn't try and collapse the ticks
    ax2.set_xticklabels(["\n$E[f(X)]$","\n$ = "+format_value(base_values, "%0.03f")+"$"], fontsize=12, ha="left")
    ax2.spines['right'].set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax2.spines['left'].set_visible(False)

    # draw the f(x) tick mark
    ax3=ax2.twiny()
    ax3.set_xlim(xmin,xmax)
    ax3.set_xticks([base_values + values.sum(), base_values + values.sum() + 1e-8]) # The 1e-8 is so matplotlib 3.3 doesn't try and collapse the ticks
    ax3.set_xticklabels(["$f(x)$","$ = "+format_value(fx, "%0.03f")+"$"], fontsize=12, ha="left")
    tick_labels = ax3.xaxis.get_majorticklabels()
    tick_labels[0].set_transform(tick_labels[0].get_transform() + matplotlib.transforms.ScaledTranslation(-10/72., 0, fig.dpi_scale_trans))
    tick_labels[1].set_transform(tick_labels[1].get_transform() + matplotlib.transforms.ScaledTranslation(12/72., 0, fig.dpi_scale_trans))
    tick_labels[1].set_color("#999999")
    ax3.spines['right'].set_visible(False)
    ax3.spines['top'].set_visible(False)
    ax3.spines['left'].set_visible(False)

    # adjust the position of the E[f(X)] = x.xx label
    tick_labels = ax2.xaxis.get_majorticklabels()
    tick_labels[0].set_transform(tick_labels[0].get_transform() + matplotlib.transforms.ScaledTranslation(-20/72., 0, fig.dpi_scale_trans))
    tick_labels[1].set_transform(tick_labels[1].get_transform() + matplotlib.transforms.ScaledTranslation(22/72., -1/72., fig.dpi_scale_trans))
    
    tick_labels[1].set_color("#999999")

    # color the y tick labels that have the feature values as gray
    # (these fall behind the black ones with just the feature name)
    tick_labels = ax.yaxis.get_majorticklabels()
    for i in range(num_features):
        tick_labels[i].set_color("#999999")
    
    if show:
        pl.show()

def dbscan_plot(data,eps=0.1,min_samples=50):
    X=data
    X = StandardScaler().fit_transform(X)
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    print('Estimated number of clusters: %d' % n_clusters_)
    print('Estimated number of noise points: %d' % n_noise_)
    print("Silhouette Coefficient: %0.3f"
          % metrics.silhouette_score(X, labels))

    # Black removed and is used for noise instead.
    plt.figure(figsize=(10, 10))
    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)
        
        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),label = k,
                 markeredgecolor='k', markersize=14)
        
        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)
    
    plt.legend(fontsize=15, title_fontsize='40')    
    plt.title('Estimated number of clusters: %d' % n_clusters_)
#    plt.show()
    return labels



def residual(params, x, data):
    alpha = params['alpha']
    beta = params['beta']
    gam = params['gamma']
 
 
    avMarkers=x['H3.3']*alpha+x['H4']*beta+x['H3']*gam
    od=x.subtract(avMarkers,axis=0)
    return np.std(od['H3.3'])+np.std(od['H4'])+np.std(od['H3'])


def residual2(params, x, data):
    beta = params['beta']
    gam = params['gamma']
 
 
    avMarkers=x['H4']*beta+x['H3.3']*gam
    od=x.subtract(avMarkers,axis=0)
    return np.std(od['H4'])+np.std(od['H3.3'])



def twoSampZ(X1, X2):
    from numpy import sqrt, abs, round
    from scipy.stats import norm
    mudiff=np.mean(X1)-np.mean(X2)
    sd1=np.std(X1)
    sd2=np.std(X2)
    n1=len(X1)
    n2=len(X2)
    pooledSE = sqrt(sd1**2/n1 + sd2**2/n2)
    z = ((X1 - X2) - mudiff)/pooledSE
    pval = 2*(1 - norm.cdf(abs(z)))
    return round(pval, 4)

def statistic(dframe):
    return dframe.corr().loc[Var1,Var2]


def draw_umap(data,n_neighbors=15, min_dist=0.1, n_components=2, metric='euclidean', title=''
              ,cc=0,rstate=42,dens=False):
    fit = umap.UMAP(
        n_neighbors=n_neighbors,
        min_dist=min_dist,
        n_components=n_components,
        metric=metric, random_state=rstate, verbose=True, densmap=dens
    )
    u = fit.fit_transform(data);
    plt.figure(figsize=(6, 5))
    if n_components == 2:
        plt.scatter(u[:,0], u[:,1], c=cc,s=3,cmap=plt.cm.seismic)
        plt.clim(-5,5)
        plt.colorbar()
    plt.title(title, fontsize=18)
    return u;


def NormMark(data):
    params = Parameters()
    params.add('beta', value=0.1, min=0)
    params.add('gamma', value=0.1, min=0)
    params.add('alpha', value=0.1, min=0)
    ddf=data.copy()
    ddf2=data.copy()
    out = minimize(residual, params, args=(ddf, ddf),method='cg')
    beta=out.params['beta'].value
    gam=out.params['gamma'].value
    alpha=out.params['alpha'].value
    avMarkers=ddf['H3.3']*alpha+ddf['H4']*beta+ddf['H3']*gam
    ddf=ddf.subtract(avMarkers,axis=0)
    data=ddf
    ddf2[EpiCols]=data[EpiCols]
#    BCKData[NamesAll]=data[NamesAll]
    data=ddf2.copy()
    del ddf
    del ddf2
    return data

def NormMark2(data):
    params = Parameters()
    params.add('beta', value=0.1, min=-1000)
    params.add('gamma', value=0.1, min=-1000)

    ddf=data.copy()
    ddf2=data.copy()
    out = minimize(residual2, params, args=(ddf, ddf),method='cg')
    beta=out.params['beta'].value
    gam=out.params['gamma'].value

    avMarkers=ddf['H4']*beta+ddf['H3.3']*gam
    ddf=ddf.subtract(avMarkers,axis=0)
    data=ddf
    ddf2[EpiCols_M]=data[EpiCols_M]
#    BCKData[NamesAll]=data[NamesAll]
    data=ddf2.copy()
    del ddf
    del ddf2
    return data






def f(): raise Exception("Found exit()")



def BPlots(data,NMS,xVar='type'):
    for NN in NMS:
        BoxVar=NN
        plt.figure(figsize=(3, 5))    
        ax = sns.boxplot(x=xVar, y=NN, data=data,showfliers=False,palette=['red','blue'])
        plt.title(NN+" MGG")
        plt.show()   

def VPlots(data,NMS,xVar='type'):
    for NN in NMS:
        BoxVar=NN
        plt.figure(figsize=(3, 5))    
        ax = sns.violinplot(x=xVar, y=NN, data=data,showfliers=False,palette=['red','blue'])
        plt.title(NN+" MGG")
        plt.show()   


def KPlots(data,NMS,titleSup=''):
    for NN in NMS:
        plt.figure(figsize=(10,10))
        sns.kdeplot(data=data,x=NN,color='blue')
        
#        plt.legend()
        plt.title(""+NN+" "+titleSup)
        plt.show()



def MeanDist(data1,data2,Markers,title='',clr=['darkgreen','purple']):
    sns.set_style({'legend.frameon':True})
 
    dd0=data1[Markers].mean().sort_values(ascending=False)
    dd1=data2[Markers].mean().sort_values()
    diffs=(dd1-dd0).sort_values(ascending=False)    

    colors = [clr[0] if x < 0 else clr[1] for x in diffs]
    
    fig, ax = plt.subplots(figsize=(16,10), dpi= 80)
    plt.hlines(y=diffs.index, xmin=0, xmax=diffs, color=colors, alpha=1, linewidth=5)
    # Decorations
    plt.gca().set(ylabel='', xlabel='')
    plt.xticks(fontsize=20 ) 
    plt.yticks(fontsize=16 ) 

    plt.title(title, fontdict={'size':20})
    plt.grid(linestyle='--', alpha=0.5)

    
def MedDist(data1,data2,Markers,title='',clr=['darkgreen','purple']):
    sns.set_style({'legend.frameon':True})
 
    dd0=data1[Markers].median().sort_values(ascending=False)
    dd1=data2[Markers].median().sort_values()
    diffs=(dd1-dd0).sort_values(ascending=False)    

    colors = [clr[0] if x < 0 else clr[1] for x in diffs]
    
    fig, ax = plt.subplots(figsize=(16,10), dpi= 80)
    plt.hlines(y=diffs.index, xmin=0, xmax=diffs, color=colors, alpha=1, linewidth=5)
    # Decorations
    plt.gca().set(ylabel='', xlabel='')
    plt.xticks(fontsize=20 ) 
    plt.yticks(fontsize=16 ) 

    plt.title(title, fontdict={'size':20})
    plt.grid(linestyle='--', alpha=0.5)    
    
def MeanDistIdU(data1,data2,Markers,title=''):
    sns.set_style({'legend.frameon':True})
 
    dd0=data1[Markers].mean().sort_values(ascending=False)
    dd1=data2[Markers].mean().sort_values()
    diffs=(dd1-dd0).sort_values(ascending=False)    
    colors = ['dodgerblue' if x < 0 else 'darkmagenta' for x in diffs]
    
    fig, ax = plt.subplots(figsize=(16,10), dpi= 80)
    plt.hlines(y=diffs.index, xmin=0, xmax=diffs, color=colors, alpha=1, linewidth=5)
    # Decorations
    plt.gca().set(ylabel='', xlabel='')
    plt.xticks(fontsize=20 ) 
    plt.yticks(fontsize=16 ) 

    plt.title(title, fontdict={'size':20})
    plt.grid(linestyle='--', alpha=0.5)

def KPlot_Mrk(Mark,titleSup=''):
    plt.figure(figsize=(10,10))
    sns.kdeplot(data=C01,x=Mark,label="C01")
    sns.kdeplot(data=C02,x=Mark,label="C02")
    sns.kdeplot(data=C03,x=Mark,label="C03")
    sns.kdeplot(data=C04,x=Mark,label="C04")
    sns.kdeplot(data=C05,x=Mark,label="C05")
    plt.legend()
    plt.title(""+Mark+" "+titleSup)
    plt.show()
    
    
    
    

def UMAP_Plot(data1,data2,Markers,Set1='C01',Set2='Other',titleSup=''):
    data1=data1.assign(Set=Set1)
    data2=data2.assign(Set=Set2)
    CAll=data1.append(data2).sample(frac=0.1).copy()
    print(CAll)
    X_2d=draw_umap(CAll[Markers],cc=CAll['H3'],min_dist=0.01)
    for NN in NamesAll:
        cc=CAll[NN]#[mask]
        plt.figure(figsize=(6, 5))
        plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                    c=cc, cmap=plt.cm.jet)
    #    cmap = matplotlib.cm.get_cmap('jet')
        plt.colorbar()
    #    plt.clim(-3.5,3.5)
        plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    #    mask=CAllmask[TSNEVar]==True
    #    rgba = cmap(-10)
    #    plt.scatter(X_2d[mask][:,0],X_2d[mask][:,1],s=2,
    #                color=rgba) 
        plt.title(NN+" "+titleSup)
        plt.show()

    plt.figure(figsize=(6, 5))
    mask=CAll.Set==Set1
    plt.scatter(X_2d[mask,0],X_2d[mask,1],s=2,
            c='blue', label=Set1)        
    mask=CAll.Set==Set2
    plt.scatter(X_2d[mask,0],X_2d[mask,1],s=2,
            c='red', label=Set2)        
    plt.legend()
    plt.show()
       

def DeltaCorr(data1,data2,Markers,titleSup=''):
    params = {'axes.titlesize': 30,
              'legend.fontsize': 20,
              'figure.figsize': (16, 10),
              'axes.labelsize': 20,
              'axes.titlesize': 20,
              'xtick.labelsize': 16,
              'ytick.labelsize': 16,
              'figure.titlesize': 30}
    plt.rcParams.update(params)
    plt.style.use('seaborn-whitegrid')
    sns.set_style("white")

    print(titleSup)
    plt.figure(figsize=(20,20))
    matrix=data2[Markers].corr()-data1[Markers].corr()
    g=sns.clustermap(matrix, annot=True, annot_kws={"size":8},
                     cmap=plt.cm.jet,vmin=matrix.min().min(),vmax=matrix.max().max(),linewidths=.1); 
    plt.xticks(rotation=0); 
    plt.yticks(rotation=0); 

    plt.title(titleSup)
    plt.show()
    
    
def DefStyle():
    params = {'axes.titlesize': 30,
          'legend.fontsize': 20,
          'figure.figsize': (6, 5),
          'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
          'figure.titlesize': 30}
    plt.rcParams.update(params)
    plt.style.use('seaborn-whitegrid')
    sns.set_style("white")

# Load and initialize

In [ ]:
NamesAll=['CD45',
 'H3',
 'K5',
 'EpCam',
 'H3K27me2',
 'p53',
 'EZH2',
 'H3K4me3',
 'gH2AX',
 'aSMA',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'Vimentin',
 'H3.3',
 'H3K64ac',
 'BMI-1',
 'ZEB1',
 'H4',
 'H3K27ac',
 'H4K20me3',
 'ER',
 'CD49f',
 'H3K36me3',
 'CD24',
 'GATA3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'CD44',
 'Ki67',
 'K8-18',
 'H3S28p',
 'Ir_DNA2',
 'Live_Dead']


EpiCols=[
 'H3',
 'H3K27me2',
 'H3K4me3',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'H3.3',
 'H3K64ac',
 'H4',
 'H3K27ac',
 'H4K20me3',
 'H3K36me3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'H3S28p',
]

CellIden=[
 'CD45',
 'K5',
 'EpCam',
 'aSMA',
 'Vimentin',
 'ZEB1',
 'ER',
 'CD49f',
 'CD24',
 'GATA3',
 'CD44',
 'K8-18',
]


ToNorm=[
 'H3',
 'K5',
 'H3K27me2',
 'p53',
 'EZH2',
 'H3K4me3',
 'gH2AX',
 'aSMA',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'Vimentin',
 'H3.3',
 'H3K64ac',
 'BMI-1',
 'ZEB1',
 'H4',
 'H3K27ac',
 'H4K20me3',
 'ER',
 'H3K36me3',
 'GATA3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'Ki67',
 'K8-18',
 'H3S28p',
 ]

NMS=['CD45',
 'K5',
 'EpCam',
 'H3K27me2',
 'p53',
 'EZH2',
 'H3K4me3',
 'gH2AX',
 'aSMA',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'Vimentin',
 'H3K64ac',
 'BMI-1',
 'ZEB1',
 'H3K27ac',
 'H4K20me3',
 'ER',
 'CD49f',
 'H3K36me3',
 'CD24',
 'GATA3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'CD44',
 'Ki67',
 'K8-18',
 'H3S28p',
]

In [ ]:
# dir="~/Dropbox/CyTOF_Breast/Kaplan_1st/"
dir="~/Desktop/biology/breast_cancer/data/"

K1=pd.read_csv(dir+"BCK-01_noaf_18Sep2022_01_0.fcs_file_internal_comp_residual.csv")
K2=pd.read_csv(dir+"BCK-02_noaf_18Sep2022_01_0.fcs_file_internal_comp_residual.csv")


params = {'axes.titlesize': 30,
          'legend.fontsize': 20,
          'figure.figsize': (6, 5),
          'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
          'figure.titlesize': 30}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

K1=K1[NamesAll]
K2=K2[NamesAll]



# Gate on H3.3/H4 too low, but also remove outliers 99.99% from all 

In [ ]:
GateColumns=['H3.3','H4']#,'H3']#,'H3']


def Gate(data,name):
    ddf=data.copy()
    print(name)
    print("Initial ",len(ddf))
    ddf=ddf[(ddf[GateColumns]>5).all(axis=1)]
    print("Core Gate ",len(ddf))
    ddf=ddf[(ddf<np.quantile(ddf,0.9999,axis=0)).all(axis=1)]
    print("Outlier Gate ",len(ddf))
    data=ddf.copy()
    del ddf
    return data


K1=Gate(K1,"K1")
K2=Gate(K2,"K2")

In [ ]:

scFac=5
K1=np.arcsinh(K1/scFac)
K2=np.arcsinh(K2/scFac)


In [ ]:
ThK1={'CD45': 1.8686868686868687}
ThK2={'CD45': 2.1717171717171717}


In [ ]:
K1CD45Neg=K1[K1.CD45<ThK1['CD45']].copy()
K2CD45Neg=K2[K2.CD45<ThK2['CD45']].copy()


In [ ]:
sns.histplot(K1.CD45,color='b',label='T1',stat='density',element='step',fill=False,)
#sns.histplot(K2.CD45,color='b',label='T2',stat='density')

In [ ]:
sns.kdeplot(K1.CD45,color='b',label='T1')
sns.kdeplot(K1CD45Neg.CD45,color='b',ls='--',label='T1 CD45-')
sns.kdeplot(K2.CD45,c='r',label='T2')
sns.kdeplot(K2CD45Neg.CD45,c='r',ls='--',label='T2 CD45-')
plt.legend(loc='upper left',bbox_to_anchor=(1,1))
plt.yscale('log')
#plt.savefig('Plots/aSMA.png',dpi=200,bbox_inches='tight')

# Normalize using new method on all intercellular markers

In [ ]:
def R(p,x,data,Q,M,M1,M2,M3):
    a=p['a']
    b=p['b']
    d=x.divide(a*M1+(1-a-b)*M2+b*M3,axis=0)
    return (d.std()['H3.3']+d.std()['H4']+d.std()['H3'])**2

def NormalizeNew(data):
    
    params = Parameters()
    params.add('a', value=0.1,min=0,max=1)
    params.add('b', value=0.1,min=0,max=1)
    ddf=data.copy()
    ddf2=data.copy()
    Q=ddf.mean()
    M=(ddf/Q)[['H3.3','H4','H3']].mean(axis=1)
    M1=(ddf/Q)['H3.3']
    M2=(ddf/Q)['H4']
    M3=(ddf/Q)['H3']

    out=minimize(R, params ,args=(ddf, ddf,Q,M,M1,M2,M3),method='cg')
    AA=out.params['a'].value
    BB=out.params['b'].value
    M=M1*AA+M2*(1-AA-BB)+M3*BB
    ddf=ddf.divide(M,axis=0).copy()
    data=ddf
    ddf2[EpiCols]=data[EpiCols]
    data=ddf2.copy()
    del ddf 
    del ddf2
    return data
    
def R2(p,x,data,Q,M,M1,M2):
    a=p['a']
    d=x.divide(a*M1+(1-a)*M2,axis=0)
    return (d.std()['H3.3'])**2+(d.std()['H4'])**2

def NormalizeNew2(data):
    
    params = Parameters()
    params.add('a', value=0.5,min=0.3,max=1)
    ddf=data.copy()
    ddf2=data.copy()
    Q=ddf.mean()
    M=(ddf/Q)[['H3.3','H4']].mean(axis=1)
    M1=(ddf/Q)['H3.3']
    M2=(ddf/Q)['H4']
 
    out=minimize(R2, params ,args=(ddf, ddf,Q,M,M1,M2),method='cg')
    AA=out.params['a'].value

    M=M1*AA+M2*(1-AA)
    ddf=ddf.divide(M,axis=0).copy()
    data=ddf.copy()
    print(data.shape,ddf2.shape)
    ddf2[EpiCols]=data[EpiCols]
    data=ddf2.copy()
    del ddf 
    del ddf2
    return data

In [ ]:
K1.std()

In [ ]:
EpiBCK=EpiCols.copy()
EpiCols=ToNorm.copy()






print("K1")
print(K1.std()['H3.3']+K1.std()['H4'])#+K1.std()['H3'])
K1=NormalizeNew2(K1)
print(K1.std()['H3.3']+K1.std()['H4'])#+K1.std()['H3'])

print("K2")
print(K2.std()['H3.3']+K2.std()['H4'])#+K2.std()['H3'])
K2=NormalizeNew2(K2)
print(K2.std()['H3.3']+K2.std()['H4'])#+K2.std()['H3'])


K1CD45Neg=NormalizeNew2(K1CD45Neg)
K2CD45Neg=NormalizeNew2(K2CD45Neg)

EpiCols=EpiBCK.copy()


In [ ]:
K1.std()

In [ ]:
sns.kdeplot(K1.EpCam)

In [ ]:
K2.columns

In [ ]:
from tqdm import tqdm
Mean_Core=K1[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K1[N]=K1[N]/Mean_Core
    
    
from tqdm import tqdm
Mean_Core=K2[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K2[N]=K2[N]/Mean_Core

In [ ]:
from tqdm import tqdm
Mean_Core=K1[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K1[N]=K1[N]/Mean_Core
    
    
from tqdm import tqdm
Mean_Core=K2[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K2[N]=K2[N]/Mean_Core

from tqdm import tqdm
Mean_Core=K1CD45Neg[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K1CD45Neg[N]=K1CD45Neg[N]/Mean_Core
    
    
from tqdm import tqdm
Mean_Core=K2CD45Neg[['H3.3','H4']].mean(axis=1)
for N in tqdm(ToNorm):
    K2CD45Neg[N]=K2CD45Neg[N]/Mean_Core

In [ ]:
sns.kdeplot(K1.CD45)
sns.kdeplot(K2.CD45)

In [ ]:
aaaa=pd.concat([K1]).copy()
m=np.mean(aaaa)
s=np.std(aaaa)
K1=(K1-m)/s

aaaa=pd.concat([K2]).copy()
m=np.mean(aaaa)
s=np.std(aaaa)
K2=(K2-m)/s

aaaa=pd.concat([K1CD45Neg]).copy()
m=np.mean(aaaa)
s=np.std(aaaa)
K1CD45Neg=(K1CD45Neg-m)/s

aaaa=pd.concat([K2CD45Neg]).copy()
m=np.mean(aaaa)
s=np.std(aaaa)
K2CD45Neg=(K2CD45Neg-m)/s


print(aaaa.std())

params = {'axes.titlesize': 30,
          'legend.fontsize': 20,
          'figure.figsize': (6, 5),
          'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
          'figure.titlesize': 30}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

In [ ]:
Lines=['Tumor1','Tumor2']

In [ ]:
print(len(K1),len(K2))
print(len(K1CD45Neg),len(K2CD45Neg))

In [ ]:
sns.kdeplot(K1CD45Neg.aSMA)

# UMAP Tumor 1 - ALL

## Cell Identity

In [ ]:

CAll=pd.concat([K1]).copy()

In [ ]:
#CAll=pd.read_csv("1.csv")

In [ ]:
CellIden

In [ ]:
#CellIden.remove('CD45')
X_2d=draw_umap(CAll[CellIden+['p53']],cc=CAll['H4'],min_dist=0.05,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("Tumor 1 Cell Iden UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_UMAP_CellIdentity_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.1,min_samples=40)

In [ ]:
m=labels!=-1

In [ ]:
sns.histplot(K1[labels!=0].CD45,color='r',element='step',fill=False,stat='density')
sns.histplot(K1.CD45,element='step',fill=False,stat='density')
plt.yscale('symlog')

In [ ]:
import scanpy as sc
import anndata

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
K1AN=K1AN[:100]

In [ ]:
sc.pp.neighbors(K1AN)
sc.tl.umap(K1AN,n_components=3)

In [ ]:
# import schist as scs
# scs.inference.nested_model(K1AN)

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:
from matplotlib.pyplot import rc_context

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=12, legend_fontoutline=2,frameon=True,
               title='Clusters Tumor 1 - Cell Iden Based', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_CellIden.png")

In [ ]:
K1AN


In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[NMS].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,20), annot_kws={"size":8}, center=0,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 Cell Iden Based') 
plt.savefig('Plots/T1_CellIden.png')

## Epigenetics Based

In [ ]:
MRK=EpiCols.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
X_2d=draw_umap(CAll[MRK],cc=CAll['H4'],min_dist=0.05,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("Tumor 1 Epigen UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_UMAP_Epi'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.2,min_samples=50)

In [ ]:
m=labels!=-1

In [ ]:
import scanpy as sc
import anndata

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
sc.pp.neighbors(K1AN)
sc.tl.umap(K1AN,n_components=3)
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:
from matplotlib.pyplot import rc_context

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=16, legend_fontoutline=4,frameon=True,
               title='Clusters Tumor 1 - Epigen Based', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_Epigen.png")

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[NMS].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,20), annot_kws={"size":8}, center=0,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 Epigen Based') 
plt.savefig('Plots/T1_Epigen.png')

## All Markers

In [ ]:
MRK=[
 'CD45',
 'K5',
 'EpCam',
 'H3K27me2',
 'p53',
 'EZH2',
 'H3K4me3',
 'gH2AX',
 'aSMA',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'Vimentin',
 'H3K64ac',
 'BMI-1',
 'ZEB1',
 'H3K27ac',
 'H4K20me3',
 'ER',
 'CD49f',
 'H3K36me3',
 'CD24',
 'GATA3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'CD44',
 'Ki67',
 'K8-18',
 'H3S28p',
]

In [ ]:

X_2d=draw_umap(CAll[MRK],cc=CAll['H4'],min_dist=0.05,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("Tumor 1 All Markers UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_UMAP_All'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.18,min_samples=50)

In [ ]:
m=labels!=-1

In [ ]:
import scanpy as sc
import anndata

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
sc.pp.neighbors(K1AN)
sc.tl.umap(K1AN,n_components=3)
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:
from matplotlib.pyplot import rc_context

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=16, legend_fontoutline=4,frameon=True,
               title='Clusters Tumor 1 - All MRK Based', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_All.png")

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[NMS].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,20), annot_kws={"size":8}, center=0,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 All MRK Based') 
plt.savefig('Plots/T1_All.png')

# UMAP Tumor 1 - CD 45 Negative

In [ ]:
K1Bck=K1.copy()
K1=K1CD45Neg.copy()

## Cell Identity

In [ ]:

CAll=pd.concat([K1]).copy()

In [ ]:
#CellIden.remove('CD45')
X_2d=draw_umap(CAll[CellIden],cc=CAll['H4'],min_dist=0.01,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("CD45- Tumor 1 Cell Iden UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_CD45Neg_UMAP_CellIdentity_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.35,min_samples=60)

In [ ]:
m=labels!=-1

In [ ]:
import scanpy as sc
import anndata

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
sc.pp.neighbors(K1AN)
#sc.tl.umap(K1AN,n_components=3)
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:
from matplotlib.pyplot import rc_context

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=16, legend_fontoutline=4,frameon=True,
               title='Clusters Tumor 1 - Cell Iden Based (CD45-)', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_CellIden_CD45Neg.png")

In [ ]:
with rc_context({'figure.figsize': (4, 4)}):
    sc.pl.umap(K1AN, color=NamesAll+['clust'],ncols=5,vmax='p99.9',vmin='p0.001',
           cmap=plt.cm.seismic,add_outline=True,show=False)
    plt.savefig("Plots/T.pdf",dpi=200,bbox_inches='tight')


In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[NMS].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,15), annot_kws={"size":12}, center=0,yticklabels=True,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 Cell Iden Based (45-)') 
plt.savefig('Plots/T1_CellIden_CD45Neg.png')

In [ ]:
EP=EpiCols.copy()
EP.remove('H3')
EP.remove('H3.3')
EP.remove('H4')

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[EP]
amin=Mat[EP].min().min()
amax=Mat[EP].max().max()
g=sns.clustermap(Mat[EP].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,15), annot_kws={"size":12}, center=0,yticklabels=True,col_cluster=False,row_cluster=True,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 Cell Iden Based (45-)') 
plt.savefig('Plots/T1_CellIden_CD45Neg_Epi.png')

In [ ]:
MeanDist(K1[K1.Clust==0],K1[K1.Clust==1],EP)

## Epigenetics Based

In [ ]:
MRK=EpiCols.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
X_2d=draw_umap(CAll[MRK],cc=CAll['H4'],min_dist=0.05,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("(CD45-) Tumor 1 Epigen UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_UMAP_Epi_CD45Neg'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.3,min_samples=50)

In [ ]:
m=labels!=-1

In [ ]:
import scanpy as sc
import anndata

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
sc.pp.neighbors(K1AN)
sc.tl.umap(K1AN,n_components=3)
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:
from matplotlib.pyplot import rc_context

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=16, legend_fontoutline=4,frameon=True,
               title='Clusters Tumor 1 - Epigen Based (CD45-)', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_Epigen_CD45Neg.png")

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[CellIden].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                 annot_kws={"size":8}, center=0,figsize=(10,20),
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 Epigen Based (CD45-)') 
plt.savefig('Plots/T1_Epigen_CD45Neg.png')

## All Markers

In [ ]:
MRK=[
 'CD45',
 'K5',
 'EpCam',
 'H3K27me2',
 'p53',
 'EZH2',
 'H3K4me3',
 'gH2AX',
 'aSMA',
 'H3K36me2',
 'H3K4me1',
 'H3K9me2',
 'H4K16ac',
 'H2Aub',
 'Vimentin',
 'H3K64ac',
 'BMI-1',
 'ZEB1',
 'H3K27ac',
 'H4K20me3',
 'ER',
 'CD49f',
 'H3K36me3',
 'CD24',
 'GATA3',
 'H3K27me3',
 'H3K9ac',
 'H3K9me3',
 'CD44',
 'Ki67',
 'K8-18',
 'H3S28p',
]

In [ ]:

X_2d=draw_umap(CAll[MRK],cc=CAll['H4'],min_dist=0.05,n_neighbors=150,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("(CD45-) Tumor 1 All Markers UMAP - "+TSNEVar)
    plt.savefig('Plots/Tumor1_UMAP_All_CD45Neg'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.17,min_samples=10)

In [ ]:
m=labels!=-1

In [ ]:
import scanpy as sc
import anndata

In [ ]:
K1AN=anndata.AnnData(K1[m],dtype=np.float32)

In [ ]:
K1['Clust']=labels

In [ ]:
sc.pp.neighbors(K1AN)
sc.tl.umap(K1AN,n_components=3)
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obsm['X_umap']=X_2d[m]

In [ ]:
K1AN.obs['clust']=K1[m].Clust.astype('category').values

In [ ]:
from matplotlib.pyplot import rc_context

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K1AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=16, legend_fontoutline=4,frameon=True,
               title='Clusters Tumor 1 - All MRK Based (CD45-)', palette=['r','orange','yellow','b'],show=False,projection='2d',)
plt.savefig("Plots/Clust_T1_All_CD45Neg.png")

In [ ]:
Mat=K1[K1.Clust!=-1].groupby(by='Clust').mean()[NMS]
amin=Mat[NMS].min().min()
amax=Mat[NMS].max().max()
g=sns.clustermap(Mat[NMS].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,20), annot_kws={"size":8}, center=0,
                annot=True, linewidths=1,linecolor='k',)
g.ax_col_dendrogram.set_title('T1 All MRK Based (CD45-)') 
plt.savefig('Plots/T1_All_CD45Neg.png')

# UMAP Tumor 2

In [ ]:
CAll=pd.concat([K2]).copy()
X_2d=draw_umap(CAll[CellIden],cc=CAll['H4'],min_dist=0.001,n_neighbors=60,rstate=42)
plt.show()

In [ ]:
for NN in ['CD45']:
    Var=NN
    TSNEVar=NN
    cc=CAllB[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2dB[:,0],X_2dB[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("Tumor 2 "+TSNEVar)
#    plt.savefig('Plots/Tumor2_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
CAll.reset_index(inplace=True)

In [ ]:
X_2dB=X_2d.copy()
CAllB=CAll.copy()

In [ ]:
X_2d=X_2dB.copy()
CAll=CAllB.copy()

In [ ]:
idx=np.random.choice(CAll.index,replace=False,size=10000)

plt.scatter(X_2d[idx,0],X_2d[idx,1])
X_2d=X_2d[idx]
CAll=CAll.iloc[idx]

In [ ]:
for NN in CellIden:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title("Tumor 2 "+TSNEVar)
#    plt.savefig('Plots/Tumor2_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.11,min_samples=100)

In [ ]:
m=labels!=-1
import scanpy as sc
import anndata

In [ ]:
K2AN=anndata.AnnData(CAll[m][CellIden],dtype=np.float32)

In [ ]:
sc.pp.neighbors(K2AN)
sc.tl.umap(K2AN)
K2AN.obsm['X_umap']=X_2d[m]

In [ ]:
CAll['Clust']=labels

In [ ]:
K2AN.obs['clust']=CAll[m].Clust.astype('category').values

In [ ]:
from matplotlib.pyplot import rc_context

with rc_context({'figure.figsize': (5, 5)}):
    sc.pl.umap(K2AN, color='clust', add_outline=True, legend_loc='on data',
               legend_fontsize=12, legend_fontoutline=2,frameon=True,
               title='Clusters Tumor 2', palette=['r','orange','yellow','g','b','indigo','magenta'],show=False)
plt.savefig("Plots/Clust_T2.png")

In [ ]:
Mat=CAll[CAll.Clust!=-1].groupby(by='Clust').mean()[NamesAll]
amin=Mat[CellIden].min().min()
amax=Mat[CellIden].max().max()
g=sns.clustermap(Mat[CellIden].T,cmap=plt.cm.seismic,vmin=amin,vmax=amax,
                figsize=(10,10), annot_kws={"size":8}, center=0,
                annot=True, linewidths=1,linecolor='k',)
plt.savefig('Plots/T2.png')

In [ ]:
sns.kdeplot(K1.CD45)
sns.kdeplot(K2.CD45)

# Bins Plot

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

In [ ]:
C05.shape[1]*[20]

In [ ]:
NamesAll

In [ ]:
MRK=NamesAll.copy()
MRK.remove('BCL6')
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
MRK.remove('Iridium')
KS=KBinsDiscretizer(n_bins=np.asarray([20]*C05.shape[1]),encode='ordinal',strategy='uniform')
C05_Binned=pd.DataFrame(KS.fit_transform(C05[NamesAll]).copy(),columns=C05.columns)
DF1=C05_Binned.groupby('BCL6').mean()
#EPC.remove('BCL6')
plt.figure()
for N in MRK:
    plt.plot(DF1.index,DF1[N],label=N,lw=3)
    plt.scatter(DF1.index,DF1[N],s=15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel('Binned BCL6')
plt.ylabel('Binned Marker')
plt.title('C05')
plt.savefig('Plots/C05_Strip.png',dpi=200,bbox_inches='tight')

In [ ]:
KS=KBinsDiscretizer(n_bins=np.asarray([20]*C06.shape[1]),encode='ordinal',strategy='uniform')
C06_Binned=pd.DataFrame(KS.fit_transform(C06[NamesAll]).copy(),columns=C06.columns)
DF1=C06_Binned.groupby('BCL6').mean()
#EPC.remove('BCL6')
plt.figure()
for N in MRK:
    plt.plot(DF1.index,DF1[N],label=N,lw=3)
    plt.scatter(DF1.index,DF1[N],s=15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel('Binned BCL6')
plt.ylabel('Binned Marker')
plt.title('C06')
plt.savefig('Plots/C06_Strip.png',dpi=200,bbox_inches='tight')

In [ ]:
KS=KBinsDiscretizer(n_bins=np.asarray([20]*C07.shape[1]),encode='ordinal',strategy='uniform')
C07_Binned=pd.DataFrame(KS.fit_transform(C07[NamesAll]).copy(),columns=C07.columns)
DF1=C07_Binned.groupby('BCL6').mean()
#EPC.remove('BCL6')
plt.figure()
for N in MRK:
    plt.plot(DF1.index,DF1[N],label=N,lw=3)
    plt.scatter(DF1.index,DF1[N],s=15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel('Binned BCL6')
plt.ylabel('Binned Marker')
plt.title('C07')
plt.savefig('Plots/C07_Strip.png',dpi=200,bbox_inches='tight')

In [ ]:
KS=KBinsDiscretizer(n_bins=np.asarray([20]*C08.shape[1]),encode='ordinal',strategy='uniform')
C08_Binned=pd.DataFrame(KS.fit_transform(C08[NamesAll]).copy(),columns=C08.columns)
DF1=C08_Binned.groupby('BCL6').mean()
#EPC.remove('BCL6')
plt.figure()
for N in MRK:
    plt.plot(DF1.index,DF1[N],label=N,lw=3)
    plt.scatter(DF1.index,DF1[N],s=15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel('Binned BCL6')
plt.ylabel('Binned Marker')
plt.title('C08')
plt.savefig('Plots/C08_Strip.png',dpi=200,bbox_inches='tight')

In [ ]:
KS=KBinsDiscretizer(n_bins=np.asarray([20]*C09.shape[1]),encode='ordinal',strategy='uniform')
C09_Binned=pd.DataFrame(KS.fit_transform(C09[NamesAll]).copy(),columns=C09.columns)
DF1=C09_Binned.groupby('BCL6').mean()
#EPC.remove('BCL6')
plt.figure()
for N in MRK:
    plt.plot(DF1.index,DF1[N],label=N,lw=3)
    plt.scatter(DF1.index,DF1[N],s=15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel('Binned BCL6')
plt.ylabel('Binned Marker')
plt.title('C09')
plt.savefig('Plots/C09_Strip.png',dpi=200,bbox_inches='tight')

In [ ]:
KS=KBinsDiscretizer(n_bins=np.asarray([20]*C10.shape[1]),encode='ordinal',strategy='uniform')
C10_Binned=pd.DataFrame(KS.fit_transform(C10[NamesAll]).copy(),columns=C10.columns)
DF1=C10_Binned.groupby('BCL6').mean()
#EPC.remove('BCL6')
plt.figure()
for N in MRK:
    plt.plot(DF1.index,DF1[N],label=N,lw=3)
    plt.scatter(DF1.index,DF1[N],s=15)
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.xlabel('Binned BCL6')
plt.ylabel('Binned Marker')
plt.title('C10')
plt.savefig('Plots/C10_Strip.png',dpi=200,bbox_inches='tight')

In [ ]:
aaa

# Rest

In [ ]:
sns.kdeplot(C05.H3K27me3,c='r',label=Lines[0])
sns.kdeplot(C06.H3K27me3,c='g',label=Lines[1])
sns.kdeplot(C07.H3K27me3,c='b',label=Lines[2])
sns.kdeplot(C08.H3K27me3,c='y',label=Lines[3])
sns.kdeplot(C09.H3K27me3,c='magenta',label=Lines[4])
sns.kdeplot(C10.H3K27me3,c='pink',label=Lines[5])
plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.xlim([-5,5])
plt.savefig('Plots/KDE_H3K27me3.png',dpi=200,bbox_inches='tight')


In [ ]:
# sns.kdeplot(C05.H3K27me3,c='r',label=Lines[0])
# sns.kdeplot(C06.H3K27me3,c='g',label=Lines[1])
# sns.kdeplot(C07.H3K27me3,c='b',label=Lines[2])
sns.kdeplot(C08.H3K27me3,c='y',label=Lines[3])
sns.kdeplot(C09.H3K27me3,c='magenta',label=Lines[4])
sns.kdeplot(C10.H3K27me3,c='pink',label=Lines[5])
plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.xlim([-5,5])
plt.savefig('Plots/KDE_H3K27me3_2.png',dpi=200,bbox_inches='tight')

In [ ]:
sns.kdeplot(C05.H3K27me3,c='r',label=Lines[0])
sns.kdeplot(C06.H3K27me3,c='g',label=Lines[1])
sns.kdeplot(C07.H3K27me3,c='b',label=Lines[2])
plt.legend(bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.xlim([-5,5])
plt.savefig('Plots/KDE_H3K27me3_3.png',dpi=200,bbox_inches='tight')

# UMAP 

In [ ]:
C05=C05.assign(Line=Lines[0])
C06=C06.assign(Line=Lines[1])
C07=C07.assign(Line=Lines[2])
C08=C08.assign(Line=Lines[3])
C09=C09.assign(Line=Lines[4])
C10=C10.assign(Line=Lines[5])

In [ ]:
EPC=EpiCols.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
CAll=pd.concat([C05.sample(n=20000,replace=True),
                C08.sample(n=20000,replace=True)]).copy()

In [ ]:
X_2d=draw_umap(CAll[EPC],cc=CAll['H4'],min_dist=0.001,n_neighbors=60,rstate=42)
plt.show()

In [ ]:
m=CAll.Line=='Ly7'
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='dodgerblue',label='Ly7')
m=CAll.Line=='EZH2'
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='purple',label='EZH2')
plt.legend(markerscale=10)
plt.savefig('Plots/UMAP_Lines_EZH_Ly7.png',dpi=200,bbox_inches='tight')

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title(" "+TSNEVar)
    plt.savefig('Plots/Ly7_EZH2_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
EPC=EpiCols.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
CAll=pd.concat([C05.sample(n=20000,replace=True),
                C06.sample(n=20000,replace=True),
                C07.sample(n=20000,replace=True),
                C08.sample(n=20000,replace=True),
                C09.sample(n=20000,replace=True),
                C10.sample(n=20000,replace=True)]).copy()

In [ ]:
X_2d=draw_umap(CAll[EPC],cc=CAll['H3K27Ac'],min_dist=0.001,n_neighbors=60,rstate=42)
plt.show()

In [ ]:
m=CAll.Line==Lines[0]
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='r',label=Lines[0])
m=CAll.Line==Lines[1]
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='g',label=Lines[1])
m=CAll.Line==Lines[2]
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='b',label=Lines[2])
m=CAll.Line==Lines[3]
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='yellow',label=Lines[3])
m=CAll.Line==Lines[4]
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='magenta',label=Lines[4])
m=CAll.Line==Lines[5]
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='pink',label=Lines[5])
plt.legend(markerscale=10)
plt.savefig('Plots/UMAP_Lines.png',dpi=200,bbox_inches='tight')

# Pride 

In [ ]:
CAll=pd.concat([C05,C06,C07,C08,C09,C10]).copy()

In [ ]:
Names=NamesAll.copy()
Names.remove('H3')
Names.remove('H3.3')
Names.remove('H4')
# Names.remove('H3K64ac')
# Names.remove('H4K16ac')

sns.set_style({'legend.frameon':True})

dd0=(C05[Names].median(axis=0)).sort_values(ascending=False)
dd1=(C06[Names].median(axis=0)).sort_values(ascending=False)
dd2=(C07[Names].median(axis=0)).sort_values(ascending=False)
dd3=(C08[Names].median(axis=0)).sort_values(ascending=False)
dd4=(C09[Names].median(axis=0)).sort_values(ascending=False)
dd5=(C10[Names].median(axis=0)).sort_values(ascending=False)

    
fig, ax = plt.subplots(figsize=(16,10), dpi= 80)
ax.hlines(y=dd0.index, xmin=-5, xmax=5, color='gray', alpha=0.7, 
          linewidth=1, linestyles='dashdot')

ax.scatter(y=dd0.index, x=dd0, s=100, c='red', alpha=0.7,
           label=Lines[0])
ax.scatter(y=dd1.index, x=dd1, s=100, c='g', alpha=0.7,
           label=Lines[1])
ax.scatter(y=dd2.index, x=dd2, s=100, c='b', alpha=0.7,
           label=Lines[2])

ax.scatter(y=dd3.index, x=dd3, s=100, c='yellow', alpha=0.7,
           label=Lines[3])

ax.scatter(y=dd4.index, x=dd4, s=100, c='magenta', alpha=0.7,
           label=Lines[4])

ax.scatter(y=dd5.index, x=dd5, s=100, c='pink', alpha=0.7,
           label=Lines[5])

ax.vlines(x=0, ymin=0, ymax=len(dd0)-1, color='black', alpha=0.7, linewidth=2, linestyles='dotted')
plt.legend(fontsize=30,
           facecolor='White', framealpha=1,frameon=True,
           bbox_to_anchor=(1.0, .50, 0.3, 0.2), loc='upper left')

ax.set_title('Median Value', fontdict={'size':30})
#ax.set_xlim(-1.5, 1.5)

labels = dd0.index.to_list()
#labels[8]="H3-K27M"
ax.set_yticklabels(labels)
ax.set_xlim([-2.5,2.5])
plt.setp(ax.get_xticklabels(), fontsize=24)
plt.setp(ax.get_yticklabels(), fontsize=24)

plt.savefig('Plots/All_Pride.png',dpi=200,bbox_inches='tight')
plt.show()

# Just Ly7/EZH2/Ly7-EZH2i/EZH2-EZH2i

In [ ]:
dir="/Users/ronguy/Dropbox/Lymphoma CyTOF/CyTOF4/"
C05=pd.read_csv(dir+"c05_Ly7.csv")
C06=pd.read_csv(dir+"c06_Ly7_EZH2i.csv")
C07=pd.read_csv(dir+"c07_Ly7_GSKJ4.csv")
C08=pd.read_csv(dir+"c08_EZH2.csv")
C09=pd.read_csv(dir+"c09_EZH2_EZH2i.csv")
C10=pd.read_csv(dir+"c10_EZH2_GSKJ4.csv")

params = {'axes.titlesize': 30,
          'legend.fontsize': 20,
          'figure.figsize': (6, 5),
          'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
          'figure.titlesize': 30}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

C05=C05[NamesAll]
C06=C06[NamesAll]
C07=C07[NamesAll]
C08=C08[NamesAll]
C09=C09[NamesAll]
C10=C10[NamesAll]



In [ ]:
sns.kdeplot(C05['H4'],c='r')
sns.kdeplot(C05['H3'],c='g')
sns.kdeplot(C05['H3.3'],c='b')


# Gate on H3.3/H4 too low, but also remove outliers 99.99% from all 

In [ ]:
GateColumns=['H3.3','H4','H3']#,'H3']#,'H3']
# #Ly7
# print("C01 Ly7")
# print(len(C01),len(C01))
# C01=C01[(C01[GateColumns]>5).all(axis=1)]
# print(len(C01),len(C01))
# C01=C01[(C01<np.quantile(C01,0.9999,axis=0)).all(axis=1)]
# print(len(C01),len(C01))

# #EZH2
# print("C03 EZH2")
# print(len(C03))
# C03=C03[(C03[GateColumns]>5).all(axis=1)]
# print(len(C03))
# C03=C03[(C03<np.quantile(C03,0.9999,axis=0)).all(axis=1)]
# print(len(C03))


def Gate(data,name):
    ddf=data.copy()
    print(name)
    print("Initial ",len(ddf))
    ddf=ddf[(ddf[GateColumns]>5).all(axis=1)]
    print("Core Gate ",len(ddf))
    ddf=ddf[(ddf<np.quantile(ddf,0.9999,axis=0)).all(axis=1)]
    print("Outlier Gate ",len(ddf))
    data=ddf.copy()
    del ddf
    return data


C05=Gate(C05,"5")
C06=Gate(C06,"")
C07=Gate(C07,"")
C08=Gate(C08,"")
C09=Gate(C09,"")
C10=Gate(C10,"")

In [ ]:

scFac=5
C05=np.arcsinh(C05/scFac)
C06=np.arcsinh(C06/scFac)
C07=np.arcsinh(C07/scFac)
C08=np.arcsinh(C08/scFac)
C09=np.arcsinh(C09/scFac)
C10=np.arcsinh(C10/scFac)

In [ ]:
EpiCols.append('BCL6')


print("C05")
print(C05.std()['H3.3']+C05.std()['H4']+C05.std()['H3'])
C05=NormalizeNew(C05)
print(C05.std()['H3.3']+C05.std()['H4']+C05.std()['H3'])

print("C06")
print(C06.std()['H3.3']+C06.std()['H4']+C06.std()['H3'])
C06=NormalizeNew(C06)
print(C06.std()['H3.3']+C06.std()['H4']+C06.std()['H3'])

print("C07")
print(C07.std()['H3.3']+C07.std()['H4']+C07.std()['H3'])
C07=NormalizeNew(C07)
print(C07.std()['H3.3']+C07.std()['H4']+C07.std()['H3'])

print("C08")
print(C08.std()['H3.3']+C08.std()['H4']+C08.std()['H3'])
C08=NormalizeNew(C08)
print(C08.std()['H3.3']+C08.std()['H4']+C08.std()['H3'])

print("C09")
print(C09.std()['H3.3']+C09.std()['H4']+C09.std()['H3'])
C09=NormalizeNew(C09)
print(C09.std()['H3.3']+C09.std()['H4']+C09.std()['H3'])

print("C05")
print(C05.std()['H3.3']+C05.std()['H4']+C05.std()['H3'])
C05=NormalizeNew(C05)
print(C05.std()['H3.3']+C05.std()['H4']+C05.std()['H3'])

print("C10")
print(C10.std()['H3.3']+C10.std()['H4']+C10.std()['H3'])
C10=NormalizeNew(C10)
print(C10.std()['H3.3']+C10.std()['H4']+C10.std()['H3'])


EpiCols.remove('BCL6')


In [ ]:
aaaa=pd.concat([C05.sample(n=20000,replace=False),
                C06.sample(n=20000,replace=False),
#                C07.sample(n=20000,replace=False),
                C08.sample(n=20000,replace=False),
                C09.sample(n=20000,replace=False),
#                C10.sample(n=20000,replace=False)]
               ]).copy()#C01.append(C02).append(C03).append(C04).append(C05).append(C06).copy()
m=np.mean(aaaa)
s=np.std(aaaa)

C05=(C05-m)/s
C06=(C06-m)/s
C07=(C07-m)/s
C08=(C08-m)/s
C09=(C09-m)/s
C10=(C10-m)/s




print(aaaa.std())

params = {'axes.titlesize': 30,
          'legend.fontsize': 20,
          'figure.figsize': (6, 5),
          'axes.labelsize': 20,
          'axes.titlesize': 20,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
          'figure.titlesize': 30}
plt.rcParams.update(params)
plt.style.use('seaborn-whitegrid')
sns.set_style("white")

In [ ]:
Lines=['Ly7','Ly7-EZH2i','Ly7-GSKJ4','EZH2','EZH2-EZH2i','EZH2-GSKJ4']

In [ ]:
C05=C05.assign(Line=Lines[0])
C06=C06.assign(Line=Lines[1])
C07=C07.assign(Line=Lines[2])
C08=C08.assign(Line=Lines[3])
C09=C09.assign(Line=Lines[4])
C10=C10.assign(Line=Lines[5])

In [ ]:
EPC=EpiCols.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
CAll=pd.concat([C05.sample(n=20000,replace=True),
                C06.sample(n=20000,replace=True),
                C08.sample(n=20000,replace=True),
                C09.sample(n=20000,replace=True),
               ]).copy()

In [ ]:
X_2d=draw_umap(CAll[EPC],cc=CAll['H3K27Ac'],min_dist=0.001,n_neighbors=60,rstate=42)
plt.show()

In [ ]:
m=CAll.Line==Lines[0]
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='r',label=Lines[0])
m=CAll.Line==Lines[1]
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='g',label=Lines[1])
m=CAll.Line==Lines[2]
#plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='b',label=Lines[2])
m=CAll.Line==Lines[3]
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='yellow',label=Lines[3])
m=CAll.Line==Lines[4]
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='magenta',label=Lines[4])
m=CAll.Line==Lines[5]
#plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='pink',label=Lines[5])
plt.legend(markerscale=10)
plt.savefig('Plots/UMAP_Lines_2.png',dpi=200,bbox_inches='tight')

In [ ]:
m1=(X_2d[:,0]<5) & (X_2d[:,1]>0) & (X_2d[:,1]<10)

In [ ]:
(m1 & (CAll.Line=='Ly7')).sum()

In [ ]:
CAll[m1].groupby(by='Line').count()/m1.sum()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title(" "+TSNEVar)
    plt.savefig('Plots/Ly7_EZH2_andInhibitors'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
BiggerEffect=(~m1 & (CAll.Line=='Ly7-EZH2i'))
SmallerEffect=(m1 & (CAll.Line=='Ly7-EZH2i'))

In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')


MeanDist(CAll[SmallerEffect],CAll[BiggerEffect],MRK,
         title='Ly7 Inhibitor Large Effect - Small Effect',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7-EZH2i_2Groups.png',dpi=200,bbox_inches='tight')


In [ ]:
MeanDist(C05,CAll[SmallerEffect],MRK,
         title='Ly7 Inhibitor Small Effect - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7-EZH2i-Small-Ly7.png',dpi=200,bbox_inches='tight')


In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')


MeanDist(C01,C02,MRK,title='KMT2D - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7_KMT2D.png',dpi=200,bbox_inches='tight')

MeanDist(C01,C03,MRK,title='EZH2 - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7_EZH2.png',dpi=200,bbox_inches='tight')

MeanDist(C01,C04,MRK,title='CREBBP - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7_CREBBP.png',dpi=200,bbox_inches='tight')

MeanDist(C01,C05,MRK,title='KMT2D/EZH2 - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7_KMT2D+EZH2.png',dpi=200,bbox_inches='tight')

MeanDist(C01,C06,MRK,title='KMT2D/CREBBP - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7_KMT2D+CREBBP.png',dpi=200,bbox_inches='tight')


# VIF

In [ ]:
VMRK=list(C01.columns)
VMRK.remove('H3.3')
VMRK.remove('H3')
VMRK.remove('H4')

from statsmodels.stats.outliers_influence import variance_inflation_factor
from tqdm import tqdm
vif_data = pd.DataFrame()
vif_data["feature"] = VMRK
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(C03[VMRK].values, i)
                          for i in tqdm(range(len(VMRK)))]

In [ ]:
vif_data.sort_values(by='VIF')

In [ ]:
pd.DataFrame(vif_data.sort_values(by='VIF')).to_excel("Plots/C03_VIF.xlsx")

In [ ]:
i=C01[VMRK].columns.get_loc("H3K27me3") 
x_i = C01[VMRK].values[:, i]
mask = np.arange(len(VMRK)) != i
x_noti = C01[VMRK].values[:, mask]

In [ ]:
i

In [ ]:
np.asarray(VMRK)[mask]

In [ ]:
from statsmodels.graphics._regressionplots_doc import _plot_influence_doc
from statsmodels.regression.linear_model import OLS

ols = OLS(x_i, x_noti).fit()


In [ ]:
Coef=pd.DataFrame()
Coef["MRK"]=np.asarray((VMRK))[mask]
Coef["Coef"]=ols.params

In [ ]:
Coef.sort_values(by='Coef',ascending=False,key=abs)

In [ ]:
plt.figure(figsize=(10,5))
ax=sns.barplot(data=Coef,x='MRK',y='Coef')
ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)


In [ ]:
def PlotInfluences(dat,Col,Tit=""):
    i=dat[VMRK].columns.get_loc(Col) 
    x_i = dat[VMRK].values[:, i]
    mask = np.arange(len(VMRK)) != i
    x_noti = dat[VMRK].values[:, mask]
    ols = OLS(x_i, x_noti).fit()
    Coef=pd.DataFrame()
    Coef["MRK"]=np.asarray((VMRK))[mask]
    Coef["Coef"]=ols.params
    plt.figure(figsize=(10,5))
    ax=sns.barplot(data=Coef,x='MRK',y='Coef')
    ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)
    plt.title("Coefficients of Regression (R2="+str(round(ols.rsquared,2))+") for "+Col+" "+Tit)
    plt.ylim([-1,1]) 

In [ ]:
for V in VMRK:
    PlotInfluences(C01,V," - C01")
#    plt.savefig('Plots/C01_Reg_'+V+'.png',dpi=200,bbox_inches='tight')
    

In [ ]:
str(round(ols.rsquared,2))


# Rest

In [ ]:
sns.ecdfplot(C03['BCL6'])

In [ ]:
C01=C01.assign(Line='Ly7')
C02=C02.assign(Line='KMT2D')
C03=C03.assign(Line='EZH2')
C04=C04.assign(Line='CREBBP')
C05=C05.assign(Line='KMT2D/EZH2')
C06=C06.assign(Line='KMT2D/CREBBP')


In [ ]:
CAll=pd.concat([C01,C02,C03,C04,C05,C06]).copy()


# KMT2D Double Mutant Analysis

In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')


MeanDist(C01,C05,MRK,title='KMT2D/ETH - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7_KMT2D+ETH_Diff.png',dpi=200,bbox_inches='tight')


In [ ]:
MeanDist(C02,C05,MRK,title='KMT2D/ETH - KMT2D',clr=['dodgerblue','orchid'])
plt.savefig('Plots/KMT2D_KMT2D+ETH_Diff.png',dpi=200,bbox_inches='tight')

In [ ]:
EPC=EpiCols.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
CAll=pd.concat([C01,C05]).copy()

In [ ]:
X_2d=draw_umap(CAll[EPC],cc=CAll['H3K27Ac'],min_dist=0.001,n_neighbors=60,rstate=42)
plt.show()

In [ ]:
m=CAll.Line=='Ly7'
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='dodgerblue')
m=CAll.Line=='KMT2D/EZH2'
plt.scatter(X_2d[m,0],X_2d[m,1],s=1,c='purple')

In [ ]:
plt.savefig('Plots/Ly7_KMT2D+ETH.png',dpi=200,bbox_inches='tight')

In [ ]:
labels=dbscan_plot(X_2d,eps=0.05,min_samples=100)

In [ ]:
labels2=labels.copy()
labels[np.isin(labels2,[0,7])]=0
labels[np.isin(labels2,[6])]=1
labels[np.isin(labels2,[3])]=2
labels[np.isin(labels2,[4])]=3
labels[np.isin(labels2,[1,5,2])]=3

In [ ]:
CAll=CAll.assign(C=labels)

In [ ]:
plt.figure(figsize=(6,5))
for i in CAll.C.unique():
    m=CAll.C==i
    plt.scatter(X_2d[m,0],X_2d[m,1],s=1,label=i)

plt.savefig('Plots/Ly7_KMT2D+ETH_Clusts.png',dpi=200,bbox_inches='tight')
plt.legend(markerscale=10,bbox_to_anchor=(1.0, 1.0), loc='upper left')

plt.show()

In [ ]:
with open('Plots/ClusteringRatios.txt', 'w') as f:
    print("Per Line by Cluster",file=f)
    print("Ly7",file=f)
    for CC in [-1,0,1,2,3]:
        m=CAll.Line=='Ly7'
        m2=CAll.C==CC
        print(f"Cluster {CC} - {(m&m2).sum()/m.sum()}",file=f) 
#        print((CAll[m].groupby(['Line']).count()/len(CAll[m]))['H3'])
#        print((CAll[m].groupby(['Line']).count()/len(CAll[m]))['H3'], file=f)
    print("KMT2D/EZH2",file=f)
    for CC in [-1,0,1,2,3]:
#        print(CC,file=f)
        m=CAll.Line=='KMT2D/EZH2'
        m2=CAll.C==CC
        print(f"Cluster {CC} - {(m&m2).sum()/m.sum()}",file=f) 
#        print((CAll[m].groupby(['Line']).count()/len(CAll[m]))['H3'])
#        print((CAll[m].groupby(['Line']).count()/len(CAll[m]))['H3'], file=f)

    print("\nPer Cluster by Line",file=f)
    for CC in [-1,0,1,2,3]:
        m=CAll.C==CC
        print(f"Cluster {CC}",file=f) 
        print((CAll[m].groupby(['Line']).count()/len(CAll[m]))['H3'])
        print((CAll[m].groupby(['Line']).count()/len(CAll[m]))['H3'], file=f)


## Diff Plots 

In [ ]:
m1=(CAll.Line=='Ly7') & (CAll.C==0)
m2=(CAll.Line=='KMT2D/EZH2') & (CAll.C==0)
MeanDist(CAll[m1],CAll[m2],MRK,title='KMT2D/ETH (WTL) - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/LY7_KMT2D+ETH_WTL_Diff.png',dpi=200,bbox_inches='tight')

In [ ]:

MeanDist(C01,C02,MRK,title='KMT2D - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7_KMT2D.png',dpi=200,bbox_inches='tight')

MeanDist(C01,C04,MRK,title='CREBBP - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7_CREBBP.png',dpi=200,bbox_inches='tight')

MeanDist(C01,C06,MRK,title='KMT2D/CREBBP - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7_MKT2D+CREBBP.png',dpi=200,bbox_inches='tight')

In [ ]:
MeanDist(C02,C06,MRK,title='KMT2D/CREBBP - CREBBP',clr=['dodgerblue','orchid'])
plt.savefig('Plots/KMT2D_KMT2D+CREBBP.png',dpi=200,bbox_inches='tight')

## Other Lines

In [ ]:
MeanDist(CAll[m1],CAll[m2],MRK,title='KMT2D/ETH Ex - WTL',clr=['dodgerblue','orchid'])
plt.savefig('Plots/LY7_KMT2D+ETH_Ex_Diff.png',dpi=200,bbox_inches='tight')

# AE Ly7/EZH2

In [ ]:
CAll=pd.concat([C01,C03])
CAll.shape

In [ ]:
EPC=EpiCols.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(CAll[EPC].values)
x_train=data_scaled

In [ ]:
import keras
from keras import layers
LE=len(EPC)
# This is the size of our encoded representations
encoding_dim = 2  # 32 floats -> compression of factor 24.5, assuming the input is 784 floats

# This is our input image
input_img = keras.Input(shape=(LE,))
# "encoded" is the encoded representation of the input
#encoded = layers.Dense(encoding_dim, activation='relu')(input_img)
# "decoded" is the lossy reconstruction of the input
#decoded = layers.Dense(len(EPC), activation='sigmoid')(encoded)


encoded = layers.Dense(np.int(LE/2), activation='relu')(input_img)
encoded = layers.Dense(LE*2, activation='relu')(encoded)
encoded = layers.Dense(8, activation='relu')(encoded)

decoded = layers.Dense(LE*2, activation='relu')(encoded)
decoded = layers.Dense(np.int(LE/2), activation='relu')(decoded)
decoded = layers.Dense(len(EPC), activation='sigmoid')(decoded)




# This model maps an input to its reconstruction
autoencoder = keras.Model(input_img, decoded)
encoder = keras.Model(input_img, encoded)
# This is our encoded (32-dimensional) input
encoded_input = keras.Input(shape=(encoding_dim,))
# Retrieve the last layer of the autoencoder model
decoder_layer = autoencoder.layers[-1]
# Create the decoder model
#decoder = keras.Model(encoded_input, decoder_layer(encoded_input))


In [ ]:
encoder.summary()

In [ ]:
from keras.callbacks import EarlyStopping

from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(x_train, x_train,
                epochs=250,
                batch_size=256,
                shuffle=True,
                validation_data=(x_train, x_train), verbose=2,
                callbacks=[es,mc])


encoder=keras.models.load_model("CyTOF3_Encoder_Ly7_EZH2.h5")
encoder.compile(optimizer='adam', loss='mse')


In [ ]:
encoded = encoder.predict(x_train)


In [ ]:
CAll.shape

In [ ]:
%matplotlib inline
X_2d=draw_umap(encoded,cc=CAll['H3K27Ac'],min_dist=0.001)

In [ ]:
Lines=['Ly7']
%matplotlib widget
idx=np.random.choice(range(len(CAll)),size=10000,replace=False)
CAll=CAll.assign(U0=X_2d[:,0])
CAll=CAll.assign(U1=X_2d[:,1])

#CAll=CAll.iloc[idx]
f=plt.figure(figsize=(6,5))
for L,c in zip(Lines,Col):
    m=CAll.Line==L
    plt.scatter(CAll[m].U0,CAll[m].U1,s=1,label=L,c=c)

plt.legend(markerscale=10,bbox_to_anchor=(1.0, 1.0), loc='upper left')

CAll=CC.copy()

pos = []
def onclick(event):
    pos.append([event.xdata,event.ydata])
f.canvas.mpl_connect('button_press_event', onclick)
f.show()


In [ ]:
%matplotlib widget
idx=np.random.choice(range(len(CAll)),size=10000,replace=False)
CAll=CAll.assign(U0=X_2d[:,0])
CAll=CAll.assign(U1=X_2d[:,1])

#CAll=CAll.iloc[idx]
f=plt.figure(figsize=(6,5))
for L,c in zip(Lines,Col):
    m=CAll.Line==L
    plt.scatter(CAll[m].U0,CAll[m].U1,s=1,label=L,c=c)

plt.legend(markerscale=10,bbox_to_anchor=(1.0, 1.0), loc='upper left')

CAll=CC.copy()

pos = []
def onclick(event):
    pos.append([event.xdata,event.ydata])
f.canvas.mpl_connect('button_press_event', onclick)
f.show()
%matplotlib inline
pos
p = Polygon(pos)

In [ ]:
Lines=['Ly7','EZH2']

idx=np.random.choice(range(len(CAll)),size=10000,replace=False)
CC=CAll.copy()
CAll=CAll.assign(U0=X_2d[:,0])
CAll=CAll.assign(U1=X_2d[:,1])
CAll=CAll.iloc[idx]
f=plt.figure(figsize=(6,5))
for L,c in zip(Lines,Col):
    m=CAll.Line==L
    plt.scatter(CAll[m].U0,CAll[m].U1,s=1,label=L,c=c)

plt.legend(markerscale=10,bbox_to_anchor=(1.0, 1.0), loc='upper left')
CAll=CC.copy()


encoder.save("CyTOF3_Encoder_Ly7_EZH2.h5")

labels=dbscan_plot(X_2d,eps=0.075,min_samples=60)

In [ ]:
CAll=CAll.assign(C=labels)

In [ ]:
for CC in CAll.C.unique():
    print(CC)
    m=CAll.C==CC
    print((CAll[m].groupby(['Line']).count()/len(CAll[m]))['H3'])

# Joint UMAP

In [ ]:
EPC=EpiCols.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
#EPC.remove('Iridium')

X_2d=draw_umap(CAll[EPC],cc=CAll['H3K27Ac'],min_dist=0.001,n_neighbors=60,rstate=42)
plt.show()

In [ ]:
Col=['r','orange','yellow','green','blue','magenta']
Lines=CAll.Line.unique()

In [ ]:
idx=np.random.choice(range(len(CAll)),size=10000,replace=False)
CC=CAll.copy()
CAll=CAll.assign(U0=X_2d[:,0])
CAll=CAll.assign(U1=X_2d[:,1])
CAll=CAll.iloc[idx]
plt.figure(figsize=(6,5))
for L,c in zip(Lines,Col):
    m=CAll.Line==L
    plt.scatter(CAll[m].U0,CAll[m].U1,s=1,label=L,c=c)

plt.legend(markerscale=10,bbox_to_anchor=(1.0, 1.0), loc='upper left')
plt.savefig('Plots/Clusts_All.png',dpi=200,bbox_inches='tight')

CAll=CC.copy()

In [ ]:
Names=EpiCols.copy()
Names.remove('H3')
Names.remove('H3.3')
Names.remove('H4')
# Names.remove('H3K64ac')
# Names.remove('H4K16ac')

sns.set_style({'legend.frameon':True})

dd0=np.mean(CAll[CAll.Line=='Ly7'][MRK]).sort_values(ascending=False)
dd1=np.mean(CAll[CAll.Line=='KMT2D'][MRK]).sort_values()
dd2=np.mean(CAll[CAll.Line=='EZH2'][MRK]).sort_values()
dd3=np.mean(CAll[CAll.Line=='CREBBP'][MRK]).sort_values()
dd4=np.mean(CAll[CAll.Line=='KTM2D/EZH2'][MRK]).sort_values()
dd5=np.mean(CAll[CAll.Line=='KMT2D/CREBBP'][MRK]).sort_values()



sz0=np.full(len(dd0),0,dtype=np.float64)
sz1=np.full(len(dd1),0,dtype=np.float64)
sz2=np.full(len(dd2),0,dtype=np.float64)

#dd0=dd0-dd0
#dd1=dd1-dd0
#dd2=dd2-dd0


LabDict={}
LabDict['Ly7']='Ly7'
LabDict['EZH2_Ex']='EZH2_Ex'
LabDict['EZH2_WTL']='EZH2_WTL'
    
fig, ax = plt.subplots(figsize=(16,10), dpi= 80)
ax.hlines(y=dd0.index, xmin=-5, xmax=5, color='gray', alpha=0.7, 
          linewidth=1, linestyles='dashdot')

ax.scatter(y=dd0.index, x=dd0, s=100, c=Col[0], alpha=0.7,
           label='Ly7',)
ax.scatter(y=dd1.index, x=dd1, s=100, c=Col[1], alpha=0.7,
           label='KMT2D',)
ax.scatter(y=dd2.index, x=dd2, s=100, c=Col[2], alpha=0.7,
           label='EZH2',)

ax.scatter(y=dd3.index, x=dd0, s=100, c=Col[3], alpha=0.7,
           label='CREBBP',)
ax.scatter(y=dd4.index, x=dd1, s=100, c=Col[4], alpha=0.7,
           label='KMT2D/EZH2',)
ax.scatter(y=dd5.index, x=dd2, s=100, c=Col[5], alpha=0.7,
           label='KMT2D/CREBBP',)



ax.vlines(x=0, ymin=0, ymax=len(dd0)-1, color='black', alpha=0.7, linewidth=2, linestyles='dotted')
plt.legend(fontsize=30,
           facecolor='White', framealpha=1,frameon=True,
           bbox_to_anchor=(1.0, .50, 0.3, 0.2), loc='upper left')

ax.set_title('Mean Value', fontdict={'size':30})
#ax.set_xlim(-1.5, 1.5)

labels = dd0.index.to_list()
#labels[8]="H3-K27M"
ax.set_yticklabels(labels)
ax.set_xlim([-2.5,2.5])
plt.setp(ax.get_xticklabels(), fontsize=24)
plt.setp(ax.get_yticklabels(), fontsize=24)

plt.savefig('Plots/All_Means.png',dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
for l,c in zip(Lines,Col):
    print(l)
    sns.kdeplot(data=CAll[CAll.Line==l],x='H3K27me2',c=c,label=l)

plt.legend()

In [ ]:
Lines

# Ly7/EZH2 UMAP

In [ ]:
CAll=pd.concat([C01,C03,C05]).sample(frac=0.25,replace=False).copy()#C01.append(C03).copy()

In [ ]:
EPC=EpiCols.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
EPC.append('BCL6')
EPC.append('CD10')
#EPC.remove('Iridium')

X_2d=draw_umap(CAll[EPC],cc=CAll['H3K27Ac'],min_dist=0.001,n_neighbors=30,rstate=42)
plt.show()

In [ ]:
idx=np.random.choice(range(len(CAll)),size=25000,replace=False)
CC=CAll.copy()
CAll=CAll.assign(U0=X_2d[:,0])
CAll=CAll.assign(U1=X_2d[:,1])
CAll=CAll.iloc[idx]

plt.figure(figsize=(6,5))
m=CAll.Line=='Ly7'
plt.scatter(CAll[m].U0,CAll[m].U1,c='dodgerblue',label='Ly7',s=1)
m=CAll.Line=='EZH2'
plt.scatter(CAll[m].U0,CAll[m].U1,c='orchid',label='EZH2',s=1)

m=CAll.Line=='KMT2D/EZH2'
plt.scatter(CAll[m].U0,CAll[m].U1,c='thistle',label='KMT2D/EZH2',s=1)


plt.legend(markerscale=10)
CAll=CC.copy()
plt.savefig('Plots/Ly7_EZH2_KMT2DEZH2_Clusters.png',dpi=200,bbox_inches='tight')


In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
%matplotlib widget
idx=np.random.choice(range(len(CAll)),size=10000,replace=False)
CAll=CAll.assign(U0=X_2d[:,0])
CAll=CAll.assign(U1=X_2d[:,1])

#CAll=CAll.iloc[idx]
f=plt.figure(figsize=(6,5))
for L,c in zip(['Ly7'],['r','b']):
    m=CAll.Line==L
    plt.scatter(CAll[m].U0,CAll[m].U1,s=1,label=L,c=c)

plt.legend(markerscale=10,bbox_to_anchor=(1.0, 1.0), loc='upper left')

CAll=CC.copy()

pos = []
def onclick(event):
    pos.append([event.xdata,event.ydata])
f.canvas.mpl_connect('button_press_event', onclick)
f.show()
#%matplotlib inline

p = Polygon(pos)

In [ ]:
%matplotlib inline
p = Polygon(pos)
idx=np.random.choice(range(len(CAll)),size=10000,replace=False)
CAll=CAll.assign(U0=X_2d[:,0])
CAll=CAll.assign(U1=X_2d[:,1])

#CAll=CAll.iloc[idx]
f=plt.figure(figsize=(6,5))
for L,c in zip(['Ly7','EZH2'],['r','b']):
    m=CAll.Line==L
    plt.scatter(CAll[m].U0,CAll[m].U1,s=1,label=L,c=c)

plt.legend(markerscale=10,bbox_to_anchor=(1.0, 1.0), loc='upper left')
x,y=p.exterior.xy
plt.plot(x,y)
CAll=CC.copy()


In [ ]:
m2=np.full_like(CAll,False)
for i in range(len(X_2d)):
    if p.contains(Point(X_2d[i,0],X_2d[i,1])):
#        print(i,X_2d[i,0],X_2d[i,1])
        m2[i]=True
    


In [ ]:
m=m2[:,0]

In [ ]:
m=m==True

In [ ]:
m2=m & (CAll.Line=='EZH2')
plt.scatter(X_2d[m2,0],X_2d[m2,1],s=1)

m2=m & (CAll.Line=='Ly7')
plt.scatter(X_2d[m2,0],X_2d[m2,1],s=1)

In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')

MeanDist(CAll[(CAll.Line=='Ly7') & m],CAll[(CAll.Line=='EZH2') & m],MRK,title='EZH2 WTL - Ly7 (Cluster 2)',clr=['dodgerblue','orchid'])

In [ ]:
labels=dbscan_plot(X_2d,eps=0.15,min_samples=60)

In [ ]:
labels2=np.full_like(labels,0)
labels2[labels==5]=1
labels2[labels==3]=2
labels2[labels==4]=3
labels2[labels==1]=4
labels2[labels==2]=4
labels2[labels==6]=4
labels2[labels==-1]=-1
CAll=CAll.assign(C=labels2)

In [ ]:
CAll=CAll.assign(C=labels2)
plt.figure()
for CC in [0,1,2,3,4]:
    m=CAll.C==CC
    plt.scatter(X_2d[m,0],X_2d[m,1],s=1,label=CC)
    
plt.legend(markerscale=10)
plt.savefig('Plots/Ly7_EZH2_Clustering.png',dpi=200,bbox_inches='tight')


In [ ]:
#CAll=CAll.assign(C=labels)
#Labs=list(CAll.C.unique())

with open('Plots/ClusteringRatios.txt', 'w') as f:
    for CC in [0,1,2,3,4]:
        print(CC)
        print(CC,file=f)
        m=CAll.C==CC
        print((CAll[m].groupby(['Line']).count()/len(CAll[m]))['H3'])
        print((CAll[m].groupby(['Line']).count()/len(CAll[m]))['H3'], file=f)

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title(" "+TSNEVar)
    plt.savefig('Plots/Ly7_EZH2_ALLMRK_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
sns.kdeplot(C03['H4'])

In [ ]:
NamekdeplotesAll.copy()
Names.remove('H3')
Names.remove('H3.3')
Names.remove('H4')
# Names.remove('H3K64ac')
# Names.remove('H4K16ac')

sns.set_style({'legend.frameon':True})

dd0=np.mean(CAll[CAll.C==0][Names]).sort_values(ascending=False)
dd1=np.mean(CAll[CAll.C==1][Names]).sort_values()
dd2=np.mean(CAll[CAll.C==2][Names]).sort_values()
dd3=np.mean(CAll[CAll.C==3][Names]).sort_values()


    
fig, ax = plt.subplots(figsize=(16,10), dpi= 80)
ax.hlines(y=dd0.index, xmin=-5, xmax=5, color='gray', alpha=0.7, 
          linewidth=1, linestyles='dashdot')

ax.scatter(y=dd0.index, x=dd0, s=100, c='blue', alpha=0.7,
           label=0)
ax.scatter(y=dd1.index, x=dd1, s=100, c='orange', alpha=0.7,
           label=1)
ax.scatter(y=dd2.index, x=dd2, s=100, c='green', alpha=0.7,
           label=2)

ax.scatter(y=dd3.index, x=dd2, s=100, c='red', alpha=0.7,
           label=3)


ax.vlines(x=0, ymin=0, ymax=len(dd0)-1, color='black', alpha=0.7, linewidth=2, linestyles='dotted')
plt.legend(fontsize=30,
           facecolor='White', framealpha=1,frameon=True,
           bbox_to_anchor=(1.0, .50, 0.3, 0.2), loc='upper left')

ax.set_title('Mean Value', fontdict={'size':30})
#ax.set_xlim(-1.5, 1.5)

labels = dd0.index.to_list()
#labels[8]="H3-K27M"
ax.set_yticklabels(labels)
ax.set_xlim([-2.5,2.5])
plt.setp(ax.get_xticklabels(), fontsize=24)
plt.setp(ax.get_yticklabels(), fontsize=24)

plt.savefig('Plots/All_Pride.png',dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')

m=(CAll.C==0)# | (CAll.C==2)

MeanDist(CAll[CAll.Line=='Ly7'],CAll[(CAll.Line=='EZH2') & m],MRK,title='EZH2 WTL (Clusters 0/2) - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Ly7_EZH2_WTL_Diff.png',dpi=200,bbox_inches='tight')


In [ ]:
CAll.C==0

In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
m=CAll.C==2
MeanDist(CAll[(CAll.Line=='Ly7') & m],CAll[(CAll.Line=='EZH2') & m],MRK,title='EZH2 WTL - Ly7 (Cluster 2)',clr=['dodgerblue','orchid'])
#plt.savefig('Plots/Ly7_EZH2_Diff.png',dpi=200,bbox_inches='tight')


# Ly7 UMAP

In [ ]:
EPC=EpiCols.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
EPC.append('BCL6')
EPC.append('CD10')
#EPC.remove('Iridium')
CAll=C01.sample(frac=0.25,replace=False).copy()
X_2d=draw_umap(CAll[EPC],cc=CAll['H3K27Ac'],min_dist=0.001,n_neighbors=100,rstate=42,dens=True)
plt.show()

In [ ]:

for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title(" "+TSNEVar)
#    plt.savefig('Plots/Ly7_ALLMRK_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
labels=dbscan_plot(X_2d,eps=0.09,min_samples=60)

In [ ]:
CAll=CAll.assign(C=labels)

In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
MRK.remove('Iridium')
plt.figure(figsize=(25,25))
sns.set(font_scale=1.2)
sns.clustermap(CAll[CAll.C==0][MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 8},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/Ly7_Corr_Clust0.png',dpi=200,bbox_inches='tight')
sns.clustermap(CAll[CAll.C==1][MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 8},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/Ly7_Corr_Clust1.png',dpi=200,bbox_inches='tight')
sns.clustermap(CAll[(CAll.C==0)|(CAll.C==1)][MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 8},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/Ly7_Corr_Clust0_1.png',dpi=200,bbox_inches='tight')


In [ ]:
DefStyle()
sns.kdeplot(data=CAll[CAll.C==0],x='BCL6',y='H3K27me3',c='r')
sns.kdeplot(data=CAll[CAll.C==1],x='BCL6',y='H3K27me3',c='b')

In [ ]:
sns.clustermap(C03[MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 8},vmin=-1,vmax=1,annot=True)

In [ ]:
Names=NamesAll.copy()
Names.remove('H3')
Names.remove('H3.3')
Names.remove('H4')
# Names.remove('H3K64ac')
# Names.remove('H4K16ac')

sns.set_style({'legend.frameon':True})

dd0=np.mean(CAll[CAll.C==0][Names]).sort_values(ascending=False)
dd1=np.mean(CAll[CAll.C==1][Names]).sort_values()
dd2=np.mean(CAll[CAll.C==2][Names]).sort_values()
dd3=np.mean(CAll[CAll.C==3][Names]).sort_values()


    
fig, ax = plt.subplots(figsize=(16,10), dpi= 80)
ax.hlines(y=dd0.index, xmin=-5, xmax=5, color='gray', alpha=0.7, 
          linewidth=1, linestyles='dashdot')

ax.scatter(y=dd0.index, x=dd0, s=100, c='blue', alpha=0.7,
           label=0)
ax.scatter(y=dd1.index, x=dd1, s=100, c='orange', alpha=0.7,
           label=1)
ax.scatter(y=dd2.index, x=dd2, s=100, c='green', alpha=0.7,
           label=2)

ax.scatter(y=dd3.index, x=dd2, s=100, c='red', alpha=0.7,
           label=3)


ax.vlines(x=0, ymin=0, ymax=len(dd0)-1, color='black', alpha=0.7, linewidth=2, linestyles='dotted')
plt.legend(fontsize=30,
           facecolor='White', framealpha=1,frameon=True,
           bbox_to_anchor=(1.0, .50, 0.3, 0.2), loc='upper left')

ax.set_title('Mean Value', fontdict={'size':30})
#ax.set_xlim(-1.5, 1.5)

labels = dd0.index.to_list()
#labels[8]="H3-K27M"
ax.set_yticklabels(labels)
ax.set_xlim([-2.5,2.5])
plt.setp(ax.get_xticklabels(), fontsize=24)
plt.setp(ax.get_yticklabels(), fontsize=24)

#plt.savefig('Plots/All_Pride.png',dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
MRK.remove('Iridium')
plt.figure(figsize=(25,25))
sns.set(font_scale=1.2)
sns.clustermap(C01[MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 8},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/Ly7_Corr.png',dpi=200,bbox_inches='tight')


In [ ]:
DefStyle()

sns.kdeplot(data=C01,x='BCL6',y='H3K27me3',s=1)
sns.kdeplot(data=C03,x='BCL6',y='H3K27me3',s=1)

In [ ]:
sns.kdeplot(C01.BCL6)

In [ ]:
m=labels==0

In [ ]:
CAll=CAll[m]
X_2d=X_2d[m]

In [ ]:

%matplotlib inline
plt.scatter(X_2d[:,0],X_2d[:,1],
            c=CAll['BCL6'],s=1,cmap=plt.cm.seismic)
plt.colorbar()

In [ ]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
polygon = Polygon([(0.63,-3.15),
                  (3.66,0.45),
                  (5.22,0.38),
                  (4.80,-03.36),
                  (0.56,-3.57)])

In [ ]:
m=CAll.BCL6>2.

In [ ]:
plt.figure(figsize=(6, 5))

plt.scatter(X_2d[:,0],X_2d[:,1],
            c=CAll['BCL6'],s=1,cmap=plt.cm.seismic)

x,y = polygon.exterior.xy
plt.plot(x,y)
plt.savefig('Plots/Ly7_BCL6_Select.png',dpi=200,bbox_inches='tight')


In [ ]:
m2=np.full_like(CAll,False)
for i in range(len(X_2d)):
    if polygon.contains(Point(X_2d[i,0],X_2d[i,1])):
#        print(i,X_2d[i,0],X_2d[i,1])
        m2[i]=True

In [ ]:
m2=m2[:,0]

In [ ]:
m2.sum()

In [ ]:
plt.figure(figsize=(6, 5))

plt.scatter(X_2d[m2==1,0],X_2d[m2==1,1],
            c='r',s=1,cmap=plt.cm.seismic)
plt.scatter(X_2d[m2==0,0],X_2d[m2==0,1],
            c='b',s=1,cmap=plt.cm.seismic)

In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
MeanDist(CAll[m2==0],CAll[m2==1],MRK,title='Ly7 BCL6 High - Ly7 Rest',clr=['red','dodgerblue'])
plt.savefig('Plots/Ly7_BCL6_Diff.png',dpi=200,bbox_inches='tight')


In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
MRK.remove('Irridium')
plt.figure(figsize=(25,25))
sns.set(font_scale=1.2)
sns.clustermap(CAll[MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 10},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/Ly7_Main_Corr.png',dpi=200,bbox_inches='tight')


In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
MRK.remove('Irridium')
plt.figure(figsize=(25,25))
sns.set(font_scale=1.2)
sns.clustermap(CAll[m2==0][MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 10},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/Ly7_Main_BCL6_Low_Corr.png',dpi=200,bbox_inches='tight')


In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
MRK.remove('Irridium')
plt.figure(figsize=(25,25))
sns.set(font_scale=1.2)
sns.clustermap(CAll[m2==1][MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 10},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/Ly7_Main_BCL6_High_Corr.png',dpi=200,bbox_inches='tight')


# Correlation HeatMaps

In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
MRK.remove('Irridium')
plt.figure(figsize=(25,25))
sns.set(font_scale=1.2)
sns.clustermap(C13[MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 10},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/Ly7_Corr.png',dpi=200,bbox_inches='tight')


In [ ]:
plt.figure(figsize=(25,25))
sns.set(font_scale=1.2)
sns.clustermap(C14[MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 10},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/EZH2_Corr.png',dpi=200,bbox_inches='tight')


# Joint UMAPs

In [ ]:
EPC=EpiCols.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
CAll=C13.append(C14).copy()
X_2d=draw_umap(CAll[EPC],cc=CAll['H3K27Ac'],min_dist=0.001,n_neighbors=60,rstate=42)
plt.show()

In [ ]:
NNN=['H3','H3.3','H4','H3K27Ac']
for NN in NNN:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title(" "+TSNEVar)
#    plt.savefig('Plots/Ly7_EZH2_NoH3_NewNorm_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

In [ ]:
idx=np.random.choice(range(len(CAll)),size=10000,replace=False)
Q=CAll.iloc[idx]
X=X_2d[idx]
m=Q.Line=='Ly7'
plt.figure()
plt.scatter(X[m,0],X[m,1],c='dodgerblue',s=1)
plt.scatter(X[~m,0],X[~m,1],c='orchid',s=1)
#plt.savefig('Plots/Ly7_EZH2_UMAP.png',dpi=200,bbox_inches='tight')

# EZH2 Assignments

In [ ]:
m=(X_2d[:,0]<1) & (CAll.Line=='EZH2')
CAll[m].mean().BCL6
EZH2_Ex=CAll[m].copy()

In [ ]:
m=(X_2d[:,0]>1) & (CAll.Line=='EZH2')
CAll[m].mean().BCL6
EZH2_WTL=CAll[m].copy()

In [ ]:
m=(X_2d[:,0]>1) & (CAll.Line=='Ly7')
CAll[m].mean().BCL6
Ly7=CAll[m].copy()

In [ ]:
sns.kdeplot(C13.CD10)
sns.kdeplot(C14.CD10)
plt.yscale('log')

In [ ]:
LVL=10#[1-0.997,1-0.96,1-0.68,0.95,0.997]
sns.kdeplot(data=C14.sample(n=1000,replace=False),x='H3K27me2',y='H3K27me3',linestyles='dashed',color='red',
           levels=LVL)
sns.kdeplot(data=Ly7.sample(n=1000,replace=False),x='H3K27me2',y='H3K27me3',color='dodgerblue',
           levels=LVL)
handles = [mpatches.Patch(facecolor='red', label="EZH2 GOF (All)"),
           mpatches.Patch(facecolor='dodgerblue', label="Ly7"),
#           mpatches.Patch(facecolor='skyblue', label="EZH2 WT Like")
          ]
plt.legend(handles=handles,
          facecolor='White', framealpha=1,frameon=True,
           bbox_to_anchor=(1.0, 1.0), loc='upper left')


plt.title('Ly7/EZH2')
#plt.savefig('Plots/C1.png',dpi=200,bbox_inches='tight')

In [ ]:
import matplotlib.patches as  mpatches
sns.kdeplot(data=Ly7.sample(n=1000,replace=False),x='H3K27me2',y='H3K27me3',color='dodgerblue',
           levels=LVL,linestyles='dashed')
sns.kdeplot(data=EZH2_Ex.sample(n=1000,replace=False),x='H3K27me2',y='H3K27me3',color='orchid',
            levels=LVL)
sns.kdeplot(data=EZH2_WTL.sample(n=1000,replace=False),x='H3K27me2',y='H3K27me3',color='skyblue',
           levels=LVL)
handles = [mpatches.Patch(facecolor='dodgerblue', label="Ly7"),
           mpatches.Patch(facecolor='orchid', label="EZH2 Extreme"),
           mpatches.Patch(facecolor='skyblue', label="EZH2 WT Like")
          ]
plt.legend(handles=handles,
          facecolor='White', framealpha=1,frameon=True,
           bbox_to_anchor=(1.0, 1.0), loc='upper left')


plt.title('EZH2 (Groups)/Ly7')
#plt.savefig('Plots/C2.png',dpi=200,bbox_inches='tight')

In [ ]:
print(Ly7.corr()['BCL6']['H3K27me3'])
print(C14.corr()['BCL6']['H3K27me3'])
print(EZH2_Ex.corr()['BCL6']['H3K27me3'])
print(EZH2_WTL.corr()['BCL6']['H3K27me3'])

In [ ]:
print(Ly7.corr()['H3K27me2']['H3K27me3'])
print(C14.corr()['H3K27me2']['H3K27me3'])
print(EZH2_Ex.corr()['H3K27me2']['H3K27me3'])
print(EZH2_WTL.corr()['H3K27me2']['H3K27me3'])

In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
plt.figure(figsize=(25,25))
sns.set(font_scale=1.2)
sns.clustermap(EZH2_WTL[MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 10},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/EZH2_WTL_Corr.png',dpi=200,bbox_inches='tight')


In [ ]:
plt.figure(figsize=(25,25))
sns.set(font_scale=1.2)
sns.clustermap(EZH2_Ex[MRK].corr(),cmap="jet", linewidths=0.2,annot_kws={"size": 10},vmin=-1,vmax=1,annot=True)
plt.savefig('Plots/EZH2_Ex_Corr.png',dpi=200,bbox_inches='tight')

In [ ]:
DefStyle()
sns.kdeplot(EZH2_Ex.H3K27me2,c='r')
sns.kdeplot(EZH2_WTL.H3K27me2,c='b')
#plt.yscale('log')

In [ ]:
len(EZH2_WTL)/(len(EZH2_WTL)+len(EZH2_Ex))

# Diff Plots


In [ ]:
MRK=NamesAll.copy()
MRK.remove('H3')
MRK.remove('H3.3')
MRK.remove('H4')
MeanDist(C13,C14,MRK,title='EZH2 - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Diff1.png',dpi=200,bbox_inches='tight')

In [ ]:
MeanDist(EZH2_WTL,EZH2_Ex,MRK,title='EZH2 Ex - EZH2 WT Like',clr=['skyblue','orchid'])
plt.savefig('Plots/Diff2.png',dpi=200,bbox_inches='tight')

In [ ]:
MeanDist(Ly7,EZH2_WTL,MRK,title='EZH2 WT Like - Ly7',clr=['dodgerblue','skyblue'])
plt.savefig('Plots/Diff3.png',dpi=200,bbox_inches='tight')

In [ ]:
MeanDist(Ly7,EZH2_Ex,MRK,title='EZH2 Ex - Ly7',clr=['dodgerblue','orchid'])
plt.savefig('Plots/Diff4.png',dpi=200,bbox_inches='tight')

In [ ]:
Names=EpiCols.copy()
Names.remove('H3')
Names.remove('H3.3')
Names.remove('H4')
# Names.remove('H3K64ac')
# Names.remove('H4K16ac')

sns.set_style({'legend.frameon':True})

dd0=np.mean(Ly7[MRK]).sort_values(ascending=False)
dd1=np.mean(EZH2_Ex[MRK]).sort_values()
dd2=np.mean(EZH2_WTL[MRK]).sort_values()

sz0=np.full(len(dd0),0,dtype=np.float64)
sz1=np.full(len(dd1),0,dtype=np.float64)
sz2=np.full(len(dd2),0,dtype=np.float64)

#dd0=dd0-dd0
dd1=dd1-dd0
dd2=dd2-dd0


LabDict={}
LabDict['Ly7']='Ly7'
LabDict['EZH2_Ex']='EZH2_Ex'
LabDict['EZH2_WTL']='EZH2_WTL'
    
fig, ax = plt.subplots(figsize=(16,10), dpi= 80)
ax.hlines(y=dd0.index, xmin=-5, xmax=5, color='gray', alpha=0.7, 
          linewidth=1, linestyles='dashdot')

ax.scatter(y=dd0.index, x=dd0, s=100, c='dodgerblue', alpha=0.7,
           label=LabDict["Ly7"],)
ax.scatter(y=dd1.index, x=dd1, s=100, c='orchid', alpha=0.7,
           label=LabDict["EZH2_Ex"],)
ax.scatter(y=dd2.index, x=dd2, s=100, c='skyblue', alpha=0.7,
           label=LabDict["EZH2_WTL"],)



ax.vlines(x=0, ymin=0, ymax=len(dd0)-1, color='black', alpha=0.7, linewidth=2, linestyles='dotted')
plt.legend(fontsize=30,
           facecolor='White', framealpha=1,frameon=True,
           bbox_to_anchor=(1.0, .50, 0.3, 0.2), loc='upper left')

ax.set_title('Mean Value', fontdict={'size':30})
#ax.set_xlim(-1.5, 1.5)

labels = dd0.index.to_list()
#labels[8]="H3-K27M"
ax.set_yticklabels(labels)
ax.set_xlim([-2.5,2.5])
plt.setp(ax.get_xticklabels(), fontsize=24)
plt.setp(ax.get_yticklabels(), fontsize=24)

plt.savefig('Plots/All_Means.png',dpi=200,bbox_inches='tight')
plt.show()

In [ ]:
sns.kdeplot(data=C14.sample(n=10000,replace=False),x='BCL6',y='H3K27me3',color='orchid',label='EZH2')
sns.kdeplot(data=C13.sample(n=10000,replace=False),x='BCL6',y='H3K27me3',color='dodgerblue',label='Ly7')
#plt.legend(markerscale=5)
plt.ylim([-4,3])
plt.xlim([-3,3])
plt.title('EZH2 (All)/Ly7')
plt.savefig('Plots/Contours_EZH2_Ly7.png',dpi=200,bbox_inches='tight')


In [ ]:
sns.kdeplot(data=EZH2_Ex.sample(n=10000,replace=False),x='BCL6',y='H3K27me3',color='orchid',label='EZH2')
sns.kdeplot(data=EZH2_WTL.sample(n=10000,replace=False),x='BCL6',y='H3K27me3',color='skyblue',label='Ly7')
sns.kdeplot(data=C13.sample(n=10000,replace=False),x='BCL6',y='H3K27me3',color='dodgerblue',label='Ly7')
#plt.legend(markerscale=5)

plt.ylim([-4,4])
plt.xlim([-4,4])
plt.savefig('Plots/Contours_EZh2_SubPop_Ly7.png',dpi=200,bbox_inches='tight')

In [ ]:
sns.kdeplot(data=C13.sample(n=10000,replace=False),x='BCL6',y='H3K27me3',color='dodgerblue',label='Ly7')
sns.kdeplot(data=Ly7.sample(n=10000,replace=False),x='BCL6',y='H3K27me3',color='blue',label='Ly7')

In [ ]:
sns.kdeplot(Ly7.BCL6)
Q=np.quantile(Ly7.BCL6,q=1-.1365)
plt.plot([Q,Q],[0,0.4])
plt.yscale('log')

In [ ]:
A=C14.sample(n=10000).assign(group='A').copy()
B=EZH2_Ex.sample(n=10000).assign(group='B').copy()
C=EZH2_WTL.sample(n=10000).assign(group='C').copy()

In [ ]:
D=A.append(B).append(C).copy()

In [ ]:
DefStyle()
sns.scatterplot(data=C14.sample(n=10000),x='BCL6',y='H3K27me3',color='r')
sns.scatterplot(data=EZH2_Ex.sample(n=10000),x='BCL6',y='H3K27me3',color='b',s=2)
sns.scatterplot(data=EZH2_WTL.sample(n=10000),x='BCL6',y='H3K27me3',color='g',s=2)

In [ ]:
DefStyle()
sns.scatterplot(data=C14.sample(n=10000),x='H3K27me2',y='H3K27me3',color='r')
sns.scatterplot(data=EZH2_Ex.sample(n=10000),x='H3K27me2',y='H3K27me3',color='b',s=2)
sns.scatterplot(data=EZH2_WTL.sample(n=10000),x='H3K27me2',y='H3K27me3',color='g',s=2)

In [ ]:
sns.kdeplot(data=EZH2_Ex.sample(n=10000,replace=False),x='BCL6',y='H3K27me3',color='orchid',label='EZH2')
sns.kdeplot(data=EZH2_WTL.sample(n=10000,replace=False),x='BCL6',y='H3K27me3',color='skyblue',label='Ly7')
sns.kdeplot(data=C14.sample(n=10000,replace=False),x='BCL6',y='H3K27me3',color='dodgerblue',label='Ly7')

In [ ]:
sns.kdeplot(data=C14.sample(n=30000,replace=False),x='BCL6',y='CD10',color='orchid',label='EZH2')
sns.kdeplot(data=C13.sample(n=30000,replace=False),x='BCL6',y='CD10',color='dodgerblue',label='Ly7')
#plt.legend(markerscale=5)

plt.savefig('Plots/Contours2.png',dpi=200,bbox_inches='tight')


In [ ]:
Q15=np.quantile(Ly7.BCL6,q=0.85)


In [ ]:
m=Ly7.BCL6>Q15

In [ ]:
MeanDist(Ly7[~m],Ly7[m],MRK,title='Ly7 High 15% BCL6 - Ly7',clr=['dodgerblue','indigo'])
#plt.savefig('Plots/Ly7_HighBCL6_vs_Rest.png',dpi=200,bbox_inches='tight')


In [ ]:
Q15=np.quantile(Ly7.CD10,q=0.15)
m=Ly7.CD10<Q15

In [ ]:
MeanDist(Ly7[~m],Ly7[m],MRK,title='Ly7 Low 15% CD10 - Ly7',clr=['dodgerblue','indigo'])
plt.savefig('Plots/Ly7_LowCD10_vs_Rest.png',dpi=200,bbox_inches='tight')


In [ ]:
sns.kdeplot(data=EZH2_Ex.sample(n=10000,replace=False),x='CD10',y='H3K27me3',color='orchid',label='EZH2')
sns.kdeplot(data=EZH2_WTL.sample(n=10000,replace=False),x='CD10',y='H3K27me3',color='skyblue',label='Ly7')
sns.kdeplot(data=Ly7.sample(n=10000,replace=False),x='CD10',y='H3K27me3',color='dodgerblue',label='Ly7')
#plt.legend(markerscale=5)

plt.ylim([-4,4])
plt.xlim([-4,4])
plt.savefig('Plots/Contours1_1.png',dpi=200,bbox_inches='tight')

In [ ]:
sns.kdeplot(data=EZH2_Ex.sample(n=10000,replace=False),x='CD10',y='H3K27Ac',color='orchid',label='EZH2')
sns.kdeplot(data=EZH2_WTL.sample(n=10000,replace=False),x='CD10',y='H3K27Ac',color='skyblue',label='Ly7')
sns.kdeplot(data=Ly7.sample(n=10000,replace=False),x='CD10',y='H3K27Ac',color='dodgerblue',label='Ly7')
#plt.legend(markerscale=5)

plt.ylim([-4,4])
plt.xlim([-4,4])
plt.savefig('Plots/Contours1_2.png',dpi=200,bbox_inches='tight')

In [ ]:
sns.kdeplot(data=EZH2_Ex.sample(n=10000,replace=False),x='BCL6',y='H3K27Ac',color='orchid',label='EZH2')
sns.kdeplot(data=EZH2_WTL.sample(n=10000,replace=False),x='BCL6',y='H3K27Ac',color='skyblue',label='Ly7')
sns.kdeplot(data=Ly7.sample(n=10000,replace=False),x='BCL6',y='H3K27Ac',color='dodgerblue',label='Ly7',zorder=0)
#plt.legend(markerscale=5)

plt.ylim([-4,4])
plt.xlim([-4,4])
plt.savefig('Plots/Contours1_3.png',dpi=200,bbox_inches='tight')

In [ ]:
#sns.kdeplot(data=EZH2_Ex.sample(n=10000,replace=False),x='BCL6',y='H3K27Ac',color='orchid',label='EZH2')
sns.kdeplot(data=EZH2_WTL.sample(n=10000,replace=False),x='BCL6',y='H3K27Ac',color='skyblue',label='Ly7')
sns.kdeplot(data=Ly7.sample(n=10000,replace=False),x='BCL6',y='H3K27Ac',color='dodgerblue',label='Ly7',zorder=0)
#plt.legend(markerscale=5)

plt.ylim([-4,4])
plt.xlim([-4,4])
plt.savefig('Plots/Contours1_4.png',dpi=200,bbox_inches='tight')

In [ ]:
sns.kdeplot(data=EZH2_Ex.sample(n=10000,replace=False),x='BCL6',y='H3K27Ac',color='orchid',label='EZH2')
sns.kdeplot(data=EZH2_WTL.sample(n=10000,replace=False),x='BCL6',y='H3K27Ac',color='skyblue',label='Ly7')
#sns.kdeplot(data=Ly7.sample(n=10000,replace=False),x='BCL6',y='H3K27Ac',color='dodgerblue',label='Ly7',zorder=0)
#plt.legend(markerscale=5)

plt.ylim([-4,4])
plt.xlim([-4,4])
plt.savefig('Plots/Contours1_5.png',dpi=200,bbox_inches='tight')

In [ ]:
# UMAP Just WTL EZH2 and WT Ly7
CAll=EZH2_WTL.append(Ly7).copy()

In [ ]:
Q=CAll.sample(frac=1,replace=False).copy()
X_2d=draw_umap(Q[EpiCols],cc=Q['H3K27Ac'],min_dist=0.00001,n_neighbors=60,rstate=42)
plt.show()

In [ ]:
m=Q.Line=='Ly7'
plt.scatter(X_2d[m,0],X_2d[m,1],c='dodgerblue',s=1,label='Ly7')

plt.scatter(X_2d[~m,0],X_2d[~m,1],c='skyblue',s=1,label='EZH2 WTL')
plt.legend(markerscale=10,bbox_to_anchor=(1.1, 1.05))
plt.title('UMAP Ly7/EZH2-WTL')

# EZH2 UMAP

In [ ]:
EPC=EpiCols.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
CAll=C14.copy()
X_2d=draw_umap(CAll[EPC],cc=CAll['H3K27Ac'],min_dist=0.001,n_neighbors=60,rstate=42)
plt.show()

In [ ]:
for NN in NamesAll:
    Var=NN
    TSNEVar=NN
    cc=CAll[NN]#[mask]
    plt.figure(figsize=(6, 5))
    plt.scatter(X_2d[:,0],X_2d[:,1],s=2,
                c=cc, cmap=plt.cm.seismic)

    plt.colorbar()

    plt.clim(cc.quantile(0.01),cc.quantile(0.99))
    plt.title(" "+TSNEVar)
    plt.savefig('Plots/EZH2_NoH3_NewNorm_'+NN+'.png',dpi=200,bbox_inches='tight')

    plt.show()

# Other

In [ ]:
print("Ly7 - STD")
print('----')
print(Ly7.std())
print("EZH2 - STD")
print('----')
print(EZH2_Ex.append(EZH2_WTL).std())


# Try clustering without UMAP - Use Agg clustering and silouette score

In [ ]:
from sklearn.metrics import silhouette_samples,silhouette_score

In [ ]:
Cls=AgglomerativeClustering(2)

In [ ]:
CAll=pd.concat([C13,C14]).sample(n=5000,replace=False).copy()

In [ ]:
EPC=NamesAll.copy()
EPC.remove('H3.3')
EPC.remove('H3')
EPC.remove('H4')
lab=Cls.fit_predict(CAll[EPC])

In [ ]:
from sklearn.cluster import MiniBatchKMeans
from sklearn.neighbors import kneighbors_graph

In [ ]:
A=kneighbors_graph(CAll[EPC],n_neighbors=10)

In [ ]:
Cls.fit_predict(CAll[EPC])

In [ ]:
T=10
x=range(4,12)
SS=np.full((T,8),0.)
for j in range(T):
    print(j)
    CAll=pd.concat([C13,C14]).sample(n=2000,replace=False).copy()
    A=kneighbors_graph(CAll[EPC],n_neighbors=10)
    for i in range (4,12):

        #Cls=MiniBatchKMeans(i)
        Cls=AgglomerativeClustering(i,connectivity=A)
        lab=Cls.fit_predict(CAll[EPC])
        s=silhouette_score(CAll[EPC],lab)
#        print(j,i-4,s)
        SS[j,i-4]=s
        

In [ ]:
plt.errorbar(x,SS.mean(axis=0),yerr=SS.std(axis=0))

In [ ]:
from castle.common import GraphDAG
from castle.metrics import MetricsDAG
from castle.datasets import IIDSimulation, DAG
from castle.algorithms import PC,Notears,GOLEM,ANMNonlinear,DirectLiNGAM,ICALiNGAM,NotearsLowRank

# data simulation, simulate true casal dag and train_data.
weighted_random_dag = DAG.erdos_renyi(n_nodes=10, n_edges=10,
                                      weight_range=(0.5, 2.0), seed=1)
dataset = IIDSimulation(W=weighted_random_dag, n=2000, method='linear',
                        sem_type='gauss')

In [ ]:
# data simulation, simulate true causal dag and train_data.

#dataset = IIDSimulation(W=weighted_random_dag, n=20000, method='linear',
                      #  sem_type='gauss')
#true_causal_matrix, X = dataset.B, dataset.X
VV=['H3K27me3','BCL6','H3K27me2','EZH2',]

VV2=[
 'H3K27me2',
 'EZH2',
 'H3K4me3',
 'pH2Ax',
 'H3K36me2',
 'H3K9me2',
 'H4K16Ac',
 'H2AK119Ub',
 'H3K4me1',
 'H3K64Ac',
 'H3K27Ac',
 'H4K20me3',
 'BCL6',
 'H3K27me3',
 'H3K9Ac',
 'H3K9me3',
 ]

# structure learning
pc = PC()
g = GOLEM()
#g.learn(X)

# plot predict_dag and true_dag
#GraphDAG(g.causal_matrix)#, true_causal_matrix)

# calculate metrics
#mt = MetricsDAG(pc.causal_matrix)#, true_causal_matrix)
#print(mt.metrics)

In [ ]:
B=np.random.normal(0,1,size=1000)+np.random.normal(0,0.1,size=1000)
C=np.random.uniform(0,1,size=1000)+np.random.normal(0,0.2,size=1000)
D=np.random.exponential(1,size=1000)+0.1*B
A=D+C+np.random.normal(0,0.5,size=1000)

In [ ]:
X=pd.DataFrame(np.asarray([A,B,C,D]).T)
X=C01[VV2].sample(n=1000,replace=False).copy()

In [ ]:
g=GOLEM(num_iter=25000,non_equal_variances=True,graph_thres=0.0,lambda_1=0,lambda_2=0)
g=Notears(w_threshold=0.01,lambda1=0.01,h_tol=1e-6)
#g=NotearsLowRank(w_threshold=0.01)
#g=ICALiNGAM(thresh=0)
g.learn(X,)#C01[VV].sample(frac=0.1,replace=False))
GraphDAG(g.causal_matrix)
AdMat=np.asarray(g.causal_matrix)

In [ ]:
import networkx as nx

In [ ]:
Col=VV2#['A','B','C','D']
G=nx.DiGraph()
for C in Col:
    G.add_node(C)

for i,Row in enumerate(AdMat):
    for j,C in enumerate(Row):
        if C==1:
            G.add_edge(Col[i],Col[j])

In [ ]:
nx.draw(G,with_labels=True)

In [ ]:
import networkx
import pydot

# See NetworkX documentation on how to build a NetworkX graph.

graph = networkx.drawing.nx_pydot.to_pydot(G)


In [ ]:
from IPython.display import Image, display

def view_pydot(pdot):
    plt = Image(pdot.create_png())
    display(plt)


In [ ]:
view_pydot(graph)

In [ ]:
Mat=[]
g=Notears(w_threshold=0.01,lambda1=0.01,h_tol=1e-5,)

for H in tqdm(range(50)):
    X=C01[VV2].sample(n=2000,replace=False).copy()
    g.learn(X,verbose=False)#C01[VV].sample(frac=0.1,replace=False))
    Mat.append(g.causal_matrix)

In [ ]:
AdMat=np.asarray(Mat).mean(axis=0)
AdMatS=np.asarray(Mat).std(axis=0)

In [ ]:
Col=VV2#['A','B','C','D']
G=nx.DiGraph()
Alph=[]
for C in Col:
    G.add_node(C)

for i,Row in enumerate(AdMat):    
    for j,C in enumerate(Row):
        S=AdMatS[i,j]
        if (C>0.1):
            if (C/S)>3:
                G.add_edge(Col[i],Col[j],penwidth=C*2,weight=C*10,color='blue',label=str(C)+'+/-'+str(round(S,2)))
    #            G.add_weighted_edges_from([(Col[i],Col[j],C)]) 
                Alph.append(C)
graph = networkx.drawing.nx_pydot.to_pydot(G)
view_pydot(graph)

In [ ]:
pos = nx.nx_pydot.pydot_layout(G, prog="dot")

In [ ]:

to_pdot = nx.drawing.nx_pydot.to_pydot
pdot = to_pdot(G)

In [ ]:
view_pydot(pdot)

In [ ]:
import sys
sys.path.insert(0, '~/Dropbox/Work/CyTOF_Lymphoma/notears')
#import notears
import notears.notears as notears

In [ ]:
num_nodes = 10
num_edges = 10
edge_coefficient_range = [0.5, 2.0]

true_adj_mat, _ = notears.utils.generate_random_dag(num_nodes, num_edges, edge_coefficient_range=edge_coefficient_range)


In [ ]:
n_sample = 1000

data = notears.utils.simulate_from_dag_lg(true_adj_mat, n_sample, mean=0, variance=1)


In [ ]:
output_dict = notears.run(notears.notears_standard, X, notears.loss.least_squares_loss, notears.loss.least_squares_loss_grad, e=1e-6, verbose=True)

In [ ]:
print('Acyclicity loss: {}'.format(output_dict['h']))
print('Least squares loss: {}'.format(output_dict['loss']))


In [ ]:
plt.matshow(output_dict['W'])
plt.title("Learned adjacency matrix")
plt.colorbar()

plt.matshow(true_adj_mat)
plt.title("True adjacency matrix")
plt.colorbar()


In [ ]:
acyclic_W = notears.utils.threshold_output(output_dict['W'])

In [ ]:
plt.matshow(output_dict['W'])
plt.title("Learned adjacency matrix")
plt.colorbar()

plt.matshow(acyclic_W)
plt.title("Learned adjacency matrix (thresholded)")
plt.colorbar()

plt.matshow(true_adj_mat)
plt.title("True adjacency matrix")
plt.colorbar()


In [ ]:
Col=VV2#['A','B','C','D']
G=nx.DiGraph()
for C in Col:
    G.add_node(C)

for i,Row in enumerate(acyclic_W):    
    for j,C in enumerate(Row):

        if C==1:

            G.add_edge(Col[i],Col[j])
            
graph = networkx.drawing.nx_pydot.to_pydot(G)
view_pydot(graph)

In [ ]:
acyclic_W.shape